[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/HumbertoDiego/3dgs-reformulated/blob/main/Torch_Backpropagation_Tests.ipynb)

#### Time everything

In [ ]:
%load_ext autotime

## Check Cuda and environment

In [ ]:
!nvidia-smi
!nvcc --version
!gcc --version
!python -c "import torch; print(torch.cuda.is_available(), torch.__version__, torch.cuda.get_device_name(0))"

# Exemplo 1

### Sobre o funcionamento da diferenciação automática do PyTorch `.autograd`:

Uma variável folha (<i>leaf</i>) é uma variável que está no início do grafo de retropropagação da diferenciação. Isso significa que nenhuma operação rastreada pelo mecanismo `.autograd` a criou. Não são rastreadas:
- Operações em tensores com `requires_grad=False`.
- Operações realizadas fora do contexto de cálculo de gradiente (como dentro de `torch.no_grad()`).
- Operações in-place (modificações diretas no tensor) podem causar problemas e não serem rastreadas corretamente.

Assim, variáveis folha é o que se deseja quando se otimiza redes neurais, pois geralmente são seus pesos ou entrada.

In [ ]:
import torch

x = torch.tensor([3.0], requires_grad=True) # x is a leaf variable

# Forward pass
y = x **2
z = y + 2
# Backward pass
z.backward()
# Print gradients
print("x is leaf?" , x.is_leaf, "-->\tx.grad=",  x.grad)
print("y is leaf?" , y.is_leaf, "-->\ty.grad=",  y.grad)

Como criar um tensor cujo gradiente NÃO seja populado pelo mecanismo `.autograd`?

In [ ]:
def info(mytensor, name='a'):
    # Forward pass
    b = mytensor**2
    # Backward pass
    b.backward()
    # Print gradients
    print(f"{name} is leaf?" , mytensor.is_leaf, f"-->\t{name}.grad=",  mytensor.grad)

def info2(mytensor, name='a'):
    # Forward pass
    b = mytensor.T @ mytensor
    # Backward pass
    b.backward()
    # Print gradients
    print(f"{name} is leaf?" , mytensor.is_leaf, f"-->\t{name}.grad=",  mytensor.grad)
    
# a is NOT a leaf variable because .cuda() is an operation creating it.
a = torch.tensor(10.0, requires_grad=True).cuda() 
info(a)

# b is NOT a leaf variable because .mean() is an operation creating it.
b = torch.rand(3, requires_grad=True).mean() 
info(b,'b')

# c[i] is NOT a leaf variable because slicing a tensor is not tracked by autograd.
c = torch.rand(3).requires_grad_()
info(c[0], 'c[0]')

# d[:] is NOT a leaf variable because slicing a tensor is not tracked by autograd.
d = torch.rand(3).requires_grad_()
info2(d[:], 'd[:]')

# b is a leaf variable.
# But e is NOT a leaf variable because it has an operation over b creating it.
# Nevertheless, e requires_grad=True by contagion, so it's gradient can reach b.
b = torch.rand(3).requires_grad_()
e = b.T @ b 
info(e, 'e')

# f is a leaf variable 
# But a view of a leaf Variable that requires grad is being used in an in-place operation.
f = torch.rand(3).requires_grad_()
try:
    f[0] = 1.0
    info2(f, 'f')
except RuntimeError as e:
    print(f"f - \033[91mError: {e}\033[0m")

# g is a leaf variable
# But b ias a new tensor, and does not preserve the computational graph of a[0,0]
# breaking the connection to a.
# So the gradient is computed with respect to this new tensor, not the original tensor a
g = torch.tensor([[1.0], [1.0], [1.0]], requires_grad=True)
h = torch.tensor([g[0,0]], requires_grad=True)
h.backward()
print("g is leaf?" , g.is_leaf, "-->\tg.grad=",  g.grad)
print("h is leaf?" , h.is_leaf, "-->\th.grad=",  h.grad)

Como criar um tensor cujo gradiente seja populado pelo mecanismo `.autograd`?

In [ ]:
# a is a leaf variable, requires grad, has no operation creating it.
a = torch.tensor(10.0).requires_grad_()
info(a)

# b is a leaf variable, requires grad, has no operation creating it.
b = torch.tensor(10.0, requires_grad=True, device="cuda") 
info(b, 'b')

# c is a leaf variable, requires grad, has no operation creating it.
c = torch.rand(3).requires_grad_()
info2(c, 'c')

# Use torch.stack() or torch.cat() to combine tensors while keeping gradient flow intact.
d = torch.tensor([1.0, 1.0, 1.0], requires_grad=True)
e = torch.stack([d[0], d[1]]).norm()
e.backward()
print("d is leaf?" , d.is_leaf, "-->\td.grad=",  d.grad)
print("e is leaf?" , e.is_leaf, "-->\te.grad=",  e.grad)


As principais operações rastreadas pelo mecanismo `.autograd` são:

1. Operações Matemáticas:
- Soma, subtração, multiplicação, divisão.
- Operações elementares como exponenciação (`torch.exp`), logaritmo (`torch.log`), seno, cosseno, etc.
- Operações de redução, como `torch.sum`, `torch.mean`, `torch.max`, etc.
2. Operações de Álgebra Linear:
- Produto escalar e matricial (`torch.matmul`, `torch.mm`).
- Decomposições como SVD, Cholesky, etc.
- Inversão de matrizes e determinantes.
3. Operações de Manipulação de Tensores
- Transposição (`torch.transpose`), reshape (`torch.view`, `torch.reshape`).
- Concatenação (`torch.cat`) e divisão (`torch.split`).
- Indexação avançada que altera o tensor.
4. Operações de Redes Neurais
- Camadas como `torch.nn.Linear`, `torch.nn.Conv2d`, etc.
- Funções de ativação como `ReLU`, `Sigmoid`, `Tanh`.
- Funções de perda como `torch.nn.CrossEntropyLoss`.
5. Operações Específicas
- Operações que envolvem `autograd`, como `torch.autograd.grad` e `torch.autograd.backward`.


Como mesclar com operações que não fazem usem do grafo de retropropagação da diferenciação do PyTorch?

$y = x^2$

$z = y^2$

$\nabla z(x) = \frac{\partial z}{\partial x} =\frac{\partial y}{\partial x} \cdot \frac{\partial z}{\partial y} = 2x \cdot 2x^2$

Para $x = 3 \rightarrow \nabla z(x) = 108$

In [ ]:
import numpy as np
import torch

def foward(x):
    return x**2

# all torch
X_torch = torch.tensor(3.0, requires_grad=True)
Y_torch = foward(X_torch)
Y_torch.retain_grad()
Z_torch = foward(Y_torch)

Z_torch.backward()

display((X_torch.grad, Y_torch.grad, Z_torch)) # 108 ,18 ,81

# all numpy
X_numpy = X_torch.detach().numpy()
Y_numpy = foward(X_numpy)
Z_numpy = foward(Y_numpy)

def backward(grad, input):
    """
    In the backward pass we receive a Tensor containing the gradient of the loss
    with respect to the output, and we need to compute the gradient of the loss
    with respect to the input.
    """
    return grad*2*input

Y_numpy_grad = backward(1, Y_numpy)
X_numpy_grad = backward(Y_numpy_grad, X_numpy)

display((X_numpy_grad, Y_numpy_grad, Z_numpy)) # 108 ,18 ,81

(tensor(108.), tensor(18.), tensor(81., grad_fn=<PowBackward0>))

(np.float32(108.0), np.float32(18.0), np.float32(81.0))

# Exemplo 2

$y = x^2$

$\nabla(x) = \frac{dy}{dx}=2x$

Para $x=3 \rightarrow \nabla(x)=6$


#### Implementação PyTorch:

In [ ]:
import torch

x = torch.tensor([3.0], requires_grad=True)

# Forward pass
y = x **2

# Backward pass
y.backward()
print(x.grad)

# Exemplo 3

$y = x  w + 1$

$\nabla(x) = \frac{\partial y}{\partial x}=w$

$\nabla(w) = \frac{\partial y}{\partial w}=x$

Para $x= 2$ e $w=3  \rightarrow$

$\nabla(x)=3$

$\nabla(w)=2$


#### Implementação PyTorch:

In [ ]:
import torch

x = torch.tensor(2.0, requires_grad=True)
w = torch.tensor(3.0, requires_grad=True)

# Forward pass
y = x * w + 1

# Backward pass
y.backward()
print(x.grad)
print(w.grad)

# Exemplo 4

$m = x_1w_1 + x_2w_2 + x_3w_3 + b$

Onde $w_i$ e $b$ são parâmetros a serem atualizados pelo gradiente descendente.

$\nabla(w_1) = \frac{\partial m}{\partial w_1}=x_1$

$\nabla(w_2) = \frac{\partial m}{\partial w_2}=x_2$

$\nabla(w_3) = \frac{\partial m}{\partial w_2}=x_3$

$\nabla(b) = \frac{\partial m}{\partial b}=1$

Para $[x_1, x_2, x_3]=[2, 3, 4]$ e quaisquer $[w_1, w_2, w_3,b] \rightarrow$

$\nabla(w_1) = 2$

$\nabla(w_2) = 3$

$\nabla(w_3) = 4$

$\nabla(b) = 1$

#### Implementação PyTorch:

In [ ]:
import torch

# Define a simple model
m = torch.nn.Linear(in_features=3, out_features=1)  # Single layer: m = xW^T + b

# Input and target
x = torch.tensor([[2.0, 3.0, 4.0]], requires_grad=True)

# Forward pass
pred = m(x)

# Backward pass
pred.backward()

# Gradients
print("Weight gradient:", m.weight.grad)
print("Bias gradient:", m.bias.grad)

# Exemplo 5

$\mathbf{x} = [x_1 , x_2 , x_3]$

$y = min([x_1^2 , x_2^2, x_3^2])$

$\nabla(\mathbf{x}) = \frac{\partial y}{\partial \mathbf{x}} = $

- Se $x_1^2$ é o mínimo:

$\frac{\partial y}{\partial \mathbf{x}} = \frac{\partial x_1^2}{\partial \mathbf{x}} = [\frac{\partial x_1^2}{\partial x_1}, \frac{\partial x_1^2}{\partial x_2}, \frac{\partial x_1^2}{\partial x_3}] = [2x_1, 0, 0 ] $

- Se $x_2^2$ é o mínimo:

$\frac{\partial y}{\partial \mathbf{x}} = \frac{\partial x_2^2}{\partial \mathbf{x}} = [\frac{\partial x_2^2}{\partial x_1}, \frac{\partial x_2^2}{\partial x_2}, \frac{\partial x_2^2}{\partial x_3}] = [0, 2x_2, 0 ] $

- Se $x_3^2$ é o mínimo:

$\frac{\partial y}{\partial \mathbf{x}} = \frac{\partial x_3^2}{\partial \mathbf{x}} = [\frac{\partial x_3^2}{\partial x_1}, \frac{\partial x_3^2}{\partial x_2}, \frac{\partial x_3^2}{\partial x_3}] = [0, 0, 2x_3 ] $

Para $x= \begin{bmatrix}0.1 & 0.8 & 0.4\end{bmatrix} \rightarrow x_1^2$ é o mínimo $\rightarrow y = 0.01$

$\nabla(x)= [2x_1, 0, 0 ] = [0.2, 0, 0 ]$

#### Implementação PyTorch:

In [ ]:
import torch

x = torch.tensor([[0.1], [0.8], [0.4]], requires_grad=True)

# Forward pass
x2 = x * x
y = x2.min(dim=0).values

# Backward pass
y.backward()

print(x.grad)

# Exemplo 6

$y = e^{(-x^2/2)}$

Onde $x$ é um parâmetro a ser atualizado pelo gradiente descendente.

$\nabla(x) = \frac{\partial y}{\partial x}= -xe^{(-x^2/2)}$

Para $[x]=[2] \rightarrow$

$\nabla(x) = -0.270670566473225$

#### Implementação SymPy:

In [ ]:
import sympy as sp

x= sp.symbols('x')
y = sp.exp(-x**2 / 2)

y_derivative = sp.diff(y, x)

print("y:")
display(y)

print("dy/dx:")
display(y_derivative)

print("Derivative evaluated at x=2:",  y_derivative.evalf(subs={x: 2}))

#### Implementação PyTorch:

In [ ]:
import torch

x = torch.tensor(2.0, requires_grad=True)

# Forward pass
y = torch.exp(-x**2 / 2)

# Backward pass
y.backward()

# Gradients
print(x.grad)

# Exemplo 7

$y = \frac{1}{\sigma\sqrt{2\pi}} e^{-0.5\frac{(x-\mu)^2}{\sigma^2}}$

Onde $x$ é um parâmetro a ser atualizado pelo gradiente descendente.

$\nabla(x) = \frac{\partial y}{\partial x}= - \frac{0.25 \sqrt{2} \left(2 x - 2 μ\right) e^{- \frac{0.5 \left(x - μ\right)^{2}}{σ^{2}}}}{\sqrt{\pi} σ^{3}}$

Para $[x, \mu, \sigma]=[2,0,1] \rightarrow$

$\nabla(x) = ? $

#### Implementação SymPy:

In [ ]:
import sympy as sp

x= sp.symbols('x')
mu = sp.symbols('μ')
sigma = sp.symbols('σ')

y = 1/(sigma * sp.sqrt(2 * sp.pi)) * sp.exp(-1/2 * (x - mu)**2 / sigma**2)

dy_dx = sp.diff(y, x)
print("Function:", y)
print("Derivative w.r.t x:", dy_dx)
print("Derivative w.r.t x evaluated at x=2, μ=0, σ=1:",  dy_dx.evalf(subs={x: 2, mu: 0, sigma: 1}))

#### Implementação PyTorch:

In [ ]:
import torch

x = torch.tensor(2.0, requires_grad=True)
mu = torch.tensor(0.0, requires_grad=False)
sigma = torch.tensor(1.0, requires_grad=False)

# Forward pass
y = 1/(sigma * torch.sqrt(torch.tensor(2.0 * torch.pi))) * torch.exp(-1/2 * (x - mu)**2 / sigma**2)

# Backward pass
y.backward()

# Gradients
print(x.grad)

# Exemplo 8

$y = \frac{1}{\sigma\sqrt{2\pi}} e^{-0.5\frac{(x-\mu)^2}{\sigma^2}}$

Onde $\mu$ e $\sigma$ são parâmetros a seres atualizados pelo gradiente descendente.

$\nabla(\mu) = \frac{\partial y}{\partial \mu}= - \frac{0.25 \sqrt{2} \left(- 2 x + 2 μ\right) e^{- \frac{0.5 \left(x - μ\right)^{2}}{σ^{2}}}}{\sqrt{\pi} σ^{3}}$

$\nabla(\sigma) = \frac{\partial y}{\partial \sigma}= - \frac{\sqrt{2} e^{- \frac{0.5 \left(x - μ\right)^{2}}{σ^{2}}}}{2 \sqrt{\pi} σ^{2}} + \frac{0.5 \sqrt{2} \left(x - μ\right)^{2} e^{- \frac{0.5 \left(x - μ\right)^{2}}{σ^{2}}}}{\sqrt{\pi} σ^{4}} $

Para $[x, \mu, \sigma]=[2,0,1] \rightarrow$

$\nabla(\mu) = ? $

$\nabla(\sigma) = ?$

#### Implementação SymPy:

In [ ]:
import sympy as sp

x= sp.symbols('x')
mu = sp.symbols('μ')
sigma = sp.symbols('σ')

y = 1/(sigma * sp.sqrt(2 * sp.pi)) * sp.exp(-1/2 * (x - mu)**2 / sigma**2)

dy_dmu = sp.diff(y, mu)
dy_dsigma = sp.diff(y, sigma)

print("Function:", y)
print("Derivative w.r.t μ:", dy_dmu)
print("Derivative w.r.t σ:", dy_dsigma)
print("Derivative w.r.t μ evaluated at x=2, μ=0, σ=1:",  dy_dmu.evalf(subs={x: 2, mu: 0, sigma: 1}))
print("Derivative w.r.t σ evaluated at x=2, μ=0, σ=1:",  dy_dsigma.evalf(subs={x: 2, mu: 0, sigma: 1}))

#### Implementação PyTorch:

In [ ]:
import torch

x = torch.tensor(2.0, requires_grad=False)
mu = torch.tensor(0.0, requires_grad=True)
sigma = torch.tensor(1.0, requires_grad=True)
# Forward pass
y = 1/(sigma * torch.sqrt(torch.tensor(2.0 * torch.pi))) * torch.exp(-1/2 * (x - mu)**2 / sigma**2)

# Backward pass
y.backward()

# Gradients
print(mu.grad)
print(sigma.grad)

# Exemplo 9

$G(x) = e^{-0.5 (\mathbf{x}-\mathbf{\mu})^T \Sigma^{-1} (\mathbf{x}-\mathbf{\mu}) }$

Onde:
- $\mathbf{\mu},\mathbf{x} \in \mathbb{R}^3$;
- $\Sigma \in \mathbb{R}^{3\times 3}$;
- $\mathbf{\mu}$ é um parâmetro a ser atualizado pelo gradiente descendente.

$\nabla(\mu) = \frac{\partial G}{\partial \mu}= \left[\frac{\partial G}{\partial \mu_1}, \frac{\partial G}{\partial \mu_2}, \frac{\partial G}{\partial \mu_3}\right] = 
\begin{bmatrix}- \frac{\left(- 0.5 x_{1} + 0.5 μ_{1}\right) \left({\Sigma}_{1,1} {\Sigma}_{2,2} - {\Sigma}_{1,2} {\Sigma}_{2,1}\right)}{{\Sigma}_{0,0} {\Sigma}_{1,1} {\Sigma}_{2,2} - {\Sigma}_{0,0} {\Sigma}_{1,2} {\Sigma}_{2,1} - {\Sigma}_{0,1} {\Sigma}_{1,0} {\Sigma}_{2,2} + {\Sigma}_{0,1} {\Sigma}_{1,2} {\Sigma}_{2,0} + {\Sigma}_{0,2} {\Sigma}_{1,0} {\Sigma}_{2,1} - {\Sigma}_{0,2} {\Sigma}_{1,1} {\Sigma}_{2,0}} + \frac{0.5 \left(x_{1} - μ_{1}\right) \left({\Sigma}_{1,1} {\Sigma}_{2,2} - {\Sigma}_{1,2} {\Sigma}_{2,1}\right)}{{\Sigma}_{0,0} {\Sigma}_{1,1} {\Sigma}_{2,2} - {\Sigma}_{0,0} {\Sigma}_{1,2} {\Sigma}_{2,1} - {\Sigma}_{0,1} {\Sigma}_{1,0} {\Sigma}_{2,2} + {\Sigma}_{0,1} {\Sigma}_{1,2} {\Sigma}_{2,0} + {\Sigma}_{0,2} {\Sigma}_{1,0} {\Sigma}_{2,1} - {\Sigma}_{0,2} {\Sigma}_{1,1} {\Sigma}_{2,0}} - \frac{\left(- 0.5 x_{2} + 0.5 μ_{2}\right) \left(- {\Sigma}_{1,0} {\Sigma}_{2,2} + {\Sigma}_{1,2} {\Sigma}_{2,0}\right)}{{\Sigma}_{0,0} {\Sigma}_{1,1} {\Sigma}_{2,2} - {\Sigma}_{0,0} {\Sigma}_{1,2} {\Sigma}_{2,1} - {\Sigma}_{0,1} {\Sigma}_{1,0} {\Sigma}_{2,2} + {\Sigma}_{0,1} {\Sigma}_{1,2} {\Sigma}_{2,0} + {\Sigma}_{0,2} {\Sigma}_{1,0} {\Sigma}_{2,1} - {\Sigma}_{0,2} {\Sigma}_{1,1} {\Sigma}_{2,0}} + \frac{0.5 \left(x_{2} - μ_{2}\right) \left(- {\Sigma}_{0,1} {\Sigma}_{2,2} + {\Sigma}_{0,2} {\Sigma}_{2,1}\right)}{{\Sigma}_{0,0} {\Sigma}_{1,1} {\Sigma}_{2,2} - {\Sigma}_{0,0} {\Sigma}_{1,2} {\Sigma}_{2,1} - {\Sigma}_{0,1} {\Sigma}_{1,0} {\Sigma}_{2,2} + {\Sigma}_{0,1} {\Sigma}_{1,2} {\Sigma}_{2,0} + {\Sigma}_{0,2} {\Sigma}_{1,0} {\Sigma}_{2,1} - {\Sigma}_{0,2} {\Sigma}_{1,1} {\Sigma}_{2,0}} - \frac{\left(- 0.5 x_{3} + 0.5 μ_{3}\right) \left({\Sigma}_{1,0} {\Sigma}_{2,1} - {\Sigma}_{1,1} {\Sigma}_{2,0}\right)}{{\Sigma}_{0,0} {\Sigma}_{1,1} {\Sigma}_{2,2} - {\Sigma}_{0,0} {\Sigma}_{1,2} {\Sigma}_{2,1} - {\Sigma}_{0,1} {\Sigma}_{1,0} {\Sigma}_{2,2} + {\Sigma}_{0,1} {\Sigma}_{1,2} {\Sigma}_{2,0} + {\Sigma}_{0,2} {\Sigma}_{1,0} {\Sigma}_{2,1} - {\Sigma}_{0,2} {\Sigma}_{1,1} {\Sigma}_{2,0}} + \frac{0.5 \left(x_{3} - μ_{3}\right) \left({\Sigma}_{0,1} {\Sigma}_{1,2} - {\Sigma}_{0,2} {\Sigma}_{1,1}\right)}{{\Sigma}_{0,0} {\Sigma}_{1,1} {\Sigma}_{2,2} - {\Sigma}_{0,0} {\Sigma}_{1,2} {\Sigma}_{2,1} - {\Sigma}_{0,1} {\Sigma}_{1,0} {\Sigma}_{2,2} + {\Sigma}_{0,1} {\Sigma}_{1,2} {\Sigma}_{2,0} + {\Sigma}_{0,2} {\Sigma}_{1,0} {\Sigma}_{2,1} - {\Sigma}_{0,2} {\Sigma}_{1,1} {\Sigma}_{2,0}}\\- \frac{\left(- 0.5 x_{1} + 0.5 μ_{1}\right) \left(- {\Sigma}_{0,1} {\Sigma}_{2,2} + {\Sigma}_{0,2} {\Sigma}_{2,1}\right)}{{\Sigma}_{0,0} {\Sigma}_{1,1} {\Sigma}_{2,2} - {\Sigma}_{0,0} {\Sigma}_{1,2} {\Sigma}_{2,1} - {\Sigma}_{0,1} {\Sigma}_{1,0} {\Sigma}_{2,2} + {\Sigma}_{0,1} {\Sigma}_{1,2} {\Sigma}_{2,0} + {\Sigma}_{0,2} {\Sigma}_{1,0} {\Sigma}_{2,1} - {\Sigma}_{0,2} {\Sigma}_{1,1} {\Sigma}_{2,0}} + \frac{0.5 \left(x_{1} - μ_{1}\right) \left(- {\Sigma}_{1,0} {\Sigma}_{2,2} + {\Sigma}_{1,2} {\Sigma}_{2,0}\right)}{{\Sigma}_{0,0} {\Sigma}_{1,1} {\Sigma}_{2,2} - {\Sigma}_{0,0} {\Sigma}_{1,2} {\Sigma}_{2,1} - {\Sigma}_{0,1} {\Sigma}_{1,0} {\Sigma}_{2,2} + {\Sigma}_{0,1} {\Sigma}_{1,2} {\Sigma}_{2,0} + {\Sigma}_{0,2} {\Sigma}_{1,0} {\Sigma}_{2,1} - {\Sigma}_{0,2} {\Sigma}_{1,1} {\Sigma}_{2,0}} - \frac{\left(- 0.5 x_{2} + 0.5 μ_{2}\right) \left({\Sigma}_{0,0} {\Sigma}_{2,2} - {\Sigma}_{0,2} {\Sigma}_{2,0}\right)}{{\Sigma}_{0,0} {\Sigma}_{1,1} {\Sigma}_{2,2} - {\Sigma}_{0,0} {\Sigma}_{1,2} {\Sigma}_{2,1} - {\Sigma}_{0,1} {\Sigma}_{1,0} {\Sigma}_{2,2} + {\Sigma}_{0,1} {\Sigma}_{1,2} {\Sigma}_{2,0} + {\Sigma}_{0,2} {\Sigma}_{1,0} {\Sigma}_{2,1} - {\Sigma}_{0,2} {\Sigma}_{1,1} {\Sigma}_{2,0}} + \frac{0.5 \left(x_{2} - μ_{2}\right) \left({\Sigma}_{0,0} {\Sigma}_{2,2} - {\Sigma}_{0,2} {\Sigma}_{2,0}\right)}{{\Sigma}_{0,0} {\Sigma}_{1,1} {\Sigma}_{2,2} - {\Sigma}_{0,0} {\Sigma}_{1,2} {\Sigma}_{2,1} - {\Sigma}_{0,1} {\Sigma}_{1,0} {\Sigma}_{2,2} + {\Sigma}_{0,1} {\Sigma}_{1,2} {\Sigma}_{2,0} + {\Sigma}_{0,2} {\Sigma}_{1,0} {\Sigma}_{2,1} - {\Sigma}_{0,2} {\Sigma}_{1,1} {\Sigma}_{2,0}} - \frac{\left(- 0.5 x_{3} + 0.5 μ_{3}\right) \left(- {\Sigma}_{0,0} {\Sigma}_{2,1} + {\Sigma}_{0,1} {\Sigma}_{2,0}\right)}{{\Sigma}_{0,0} {\Sigma}_{1,1} {\Sigma}_{2,2} - {\Sigma}_{0,0} {\Sigma}_{1,2} {\Sigma}_{2,1} - {\Sigma}_{0,1} {\Sigma}_{1,0} {\Sigma}_{2,2} + {\Sigma}_{0,1} {\Sigma}_{1,2} {\Sigma}_{2,0} + {\Sigma}_{0,2} {\Sigma}_{1,0} {\Sigma}_{2,1} - {\Sigma}_{0,2} {\Sigma}_{1,1} {\Sigma}_{2,0}} + \frac{0.5 \left(x_{3} - μ_{3}\right) \left(- {\Sigma}_{0,0} {\Sigma}_{1,2} + {\Sigma}_{0,2} {\Sigma}_{1,0}\right)}{{\Sigma}_{0,0} {\Sigma}_{1,1} {\Sigma}_{2,2} - {\Sigma}_{0,0} {\Sigma}_{1,2} {\Sigma}_{2,1} - {\Sigma}_{0,1} {\Sigma}_{1,0} {\Sigma}_{2,2} + {\Sigma}_{0,1} {\Sigma}_{1,2} {\Sigma}_{2,0} + {\Sigma}_{0,2} {\Sigma}_{1,0} {\Sigma}_{2,1} - {\Sigma}_{0,2} {\Sigma}_{1,1} {\Sigma}_{2,0}}\\- \frac{\left(- 0.5 x_{1} + 0.5 μ_{1}\right) \left({\Sigma}_{0,1} {\Sigma}_{1,2} - {\Sigma}_{0,2} {\Sigma}_{1,1}\right)}{{\Sigma}_{0,0} {\Sigma}_{1,1} {\Sigma}_{2,2} - {\Sigma}_{0,0} {\Sigma}_{1,2} {\Sigma}_{2,1} - {\Sigma}_{0,1} {\Sigma}_{1,0} {\Sigma}_{2,2} + {\Sigma}_{0,1} {\Sigma}_{1,2} {\Sigma}_{2,0} + {\Sigma}_{0,2} {\Sigma}_{1,0} {\Sigma}_{2,1} - {\Sigma}_{0,2} {\Sigma}_{1,1} {\Sigma}_{2,0}} + \frac{0.5 \left(x_{1} - μ_{1}\right) \left({\Sigma}_{1,0} {\Sigma}_{2,1} - {\Sigma}_{1,1} {\Sigma}_{2,0}\right)}{{\Sigma}_{0,0} {\Sigma}_{1,1} {\Sigma}_{2,2} - {\Sigma}_{0,0} {\Sigma}_{1,2} {\Sigma}_{2,1} - {\Sigma}_{0,1} {\Sigma}_{1,0} {\Sigma}_{2,2} + {\Sigma}_{0,1} {\Sigma}_{1,2} {\Sigma}_{2,0} + {\Sigma}_{0,2} {\Sigma}_{1,0} {\Sigma}_{2,1} - {\Sigma}_{0,2} {\Sigma}_{1,1} {\Sigma}_{2,0}} - \frac{\left(- 0.5 x_{2} + 0.5 μ_{2}\right) \left(- {\Sigma}_{0,0} {\Sigma}_{1,2} + {\Sigma}_{0,2} {\Sigma}_{1,0}\right)}{{\Sigma}_{0,0} {\Sigma}_{1,1} {\Sigma}_{2,2} - {\Sigma}_{0,0} {\Sigma}_{1,2} {\Sigma}_{2,1} - {\Sigma}_{0,1} {\Sigma}_{1,0} {\Sigma}_{2,2} + {\Sigma}_{0,1} {\Sigma}_{1,2} {\Sigma}_{2,0} + {\Sigma}_{0,2} {\Sigma}_{1,0} {\Sigma}_{2,1} - {\Sigma}_{0,2} {\Sigma}_{1,1} {\Sigma}_{2,0}} + \frac{0.5 \left(x_{2} - μ_{2}\right) \left(- {\Sigma}_{0,0} {\Sigma}_{2,1} + {\Sigma}_{0,1} {\Sigma}_{2,0}\right)}{{\Sigma}_{0,0} {\Sigma}_{1,1} {\Sigma}_{2,2} - {\Sigma}_{0,0} {\Sigma}_{1,2} {\Sigma}_{2,1} - {\Sigma}_{0,1} {\Sigma}_{1,0} {\Sigma}_{2,2} + {\Sigma}_{0,1} {\Sigma}_{1,2} {\Sigma}_{2,0} + {\Sigma}_{0,2} {\Sigma}_{1,0} {\Sigma}_{2,1} - {\Sigma}_{0,2} {\Sigma}_{1,1} {\Sigma}_{2,0}} - \frac{\left(- 0.5 x_{3} + 0.5 μ_{3}\right) \left({\Sigma}_{0,0} {\Sigma}_{1,1} - {\Sigma}_{0,1} {\Sigma}_{1,0}\right)}{{\Sigma}_{0,0} {\Sigma}_{1,1} {\Sigma}_{2,2} - {\Sigma}_{0,0} {\Sigma}_{1,2} {\Sigma}_{2,1} - {\Sigma}_{0,1} {\Sigma}_{1,0} {\Sigma}_{2,2} + {\Sigma}_{0,1} {\Sigma}_{1,2} {\Sigma}_{2,0} + {\Sigma}_{0,2} {\Sigma}_{1,0} {\Sigma}_{2,1} - {\Sigma}_{0,2} {\Sigma}_{1,1} {\Sigma}_{2,0}} + \frac{0.5 \left(x_{3} - μ_{3}\right) \left({\Sigma}_{0,0} {\Sigma}_{1,1} - {\Sigma}_{0,1} {\Sigma}_{1,0}\right)}{{\Sigma}_{0,0} {\Sigma}_{1,1} {\Sigma}_{2,2} - {\Sigma}_{0,0} {\Sigma}_{1,2} {\Sigma}_{2,1} - {\Sigma}_{0,1} {\Sigma}_{1,0} {\Sigma}_{2,2} + {\Sigma}_{0,1} {\Sigma}_{1,2} {\Sigma}_{2,0} + {\Sigma}_{0,2} {\Sigma}_{1,0} {\Sigma}_{2,1} - {\Sigma}_{0,2} {\Sigma}_{1,1} {\Sigma}_{2,0}}\end{bmatrix}^T$

Para $\mathbf{x}=[2,2,3],\  \mathbf{\mu}=[1,1,1] \ $ e $\Sigma = I \rightarrow$

$\nabla(\mu) = ? $

#### Implementação SymPy:

In [ ]:
import sympy as sp

x1, x2, x3= sp.symbols('x_1 x_2 x_3')
mu1, mu2, mu3 = sp.symbols('μ_1 μ_2 μ_3')
x = sp.Matrix([x1, x2, x3])
mu = sp.Matrix([mu1, mu2, mu3])
Sigma = sp.Matrix([[1,0,0],[0,1,0],[0,0,1]])

G = sp.exp(-0.5 * (x - mu).T * Sigma.inv() * (x - mu)) 

dG_dmu = sp.diff(G, mu)

print("G:")
display(G)
print("Derivative of G w.r.t μ (dG/dmu):")
display(sp.simplify(dG_dmu))
dG_dmu_num = dG_dmu.subs({x1:2, x2:2, x3:3, mu1:1, mu2:1, mu3:1}).tolist()
print("Derivative w.r.t μ evaluated at x=[2,2,3], μ=[1,1,1]:\n", dG_dmu_num )

#### Implementação PyTorch:

In [ ]:
import torch

x = torch.tensor([[2.0], [2.0], [3.0]], requires_grad=True)
mu = torch.tensor([[1.0], [1.0], [1.0]], requires_grad=True)
Sigma = torch.eye(3)

# Forward pass
dx = x - mu
g = torch.exp(-0.5 * dx.T @ torch.inverse(Sigma) @ dx)

# Backward pass
g.backward()

# Gradients
print(mu.grad)

#### Comparação:

In [ ]:
dg_dmu = torch.tensor(dG_dmu_num, dtype=torch.float32).reshape(mu.grad.shape)

# Gradients
print(mu.grad,"=\n", dg_dmu)

# Exemplo 10

$\displaystyle C(\mathbf{p}) = \sum_{i=1}^{N} \left[ c_i \ G_i(\mathbf{p})  \prod_{j=1}^{i-1} (1- G_j(\mathbf{p})) \right]$

Onde:
- $\displaystyle G_i(\mathbf{p}) = exp(-0.5 (\mathbf{p}-\mathbf{\mu_i})^T \Sigma_i^{'-1} (\mathbf{p}-\mathbf{\mu_i}))$;
- $c_i \in \mathbb{R}$;
- $\mathbf{\mu_i},\mathbf{p} \in \mathbb{R}^2$;
- $\Sigma_i' \in \mathbb{R}^{2\times 2}$;
- $\mathbf{\mu_i}$ são parâmetros a ser atualizados pelo gradiente descendente.

$\displaystyle \nabla(\mu) = \frac{\partial C}{\partial \mu} = \sum_{i=1}^{N} \left[ c_i \ \frac{\partial G_i}{\partial \mu} \prod_{j=1}^{i-1} (1- G_j)  + c_i G_i \sum_{k=1}^{i-1} \left(\prod_{j=1}^{k-1}  (1- G_j)\right) \left(-\frac{\partial G_k}{\partial \mu}\right) \left(\prod_{j=k+1}^{i-1}  (1- G_j)\right) \right]$

Para $N=3, \mathbf{p}=[2,3],\ c_i=1/(i+1),\ \mathbf{\mu_i}=[1+i,1+i] \ $ e $\Sigma_i^{'-1} = I \rightarrow$

$\nabla(\mu) = ?$

#### Implementação SymPy:

In [ ]:
import sympy as sp

N = 3  # Number of components
Sigma_inv_i = sp.Matrix([[1.0, 0.0], [0.0, 1.0]]) # assumir conhecida por simplicidade
Sigma_inv_j = sp.Matrix([[1.0, 0.0], [0.0, 1.0]])

# variam para n=1,...,N
c = sp.IndexedBase('c')
mux = sp.IndexedBase('μ_x')
muy = sp.IndexedBase('μ_y')

# Ponto de pesquisa
px, py= sp.symbols('p_x p_y')
p = sp.Matrix([px, py])

# Generate values to apply to the derivative
values = {px:2, py:3}
for i in range(N):
    values = values | {mux[i]:1+i, muy[i]:1+i, c[i]:1/(i+1)}

# Expression for C(x)
C = 0 
for i in range(0,N):
    d_i = p - sp.Matrix([mux[i], muy[i]])
    G_i = sp.exp(-0.5 * (d_i.T * Sigma_inv_i * d_i)[0, 0])
    prod = 1
    for j in range(0, i):
        d_j = p - sp.Matrix([mux[j], muy[j]])
        G_j = sp.exp(-0.5 * (d_j.T * Sigma_inv_j * d_j)[0, 0])
        prod *= (1 - G_j)
    C += c[i] * G_i * prod

print("C(p)=")
display(C)
print(f"C(p) evaluated at {values}:", C.subs(values).evalf())

dC_dmu =[]
for i in range(N):
    dC_dmu_i = sp.diff(C, sp.Matrix([mux[i], muy[i]]))
    dC_dmu_num = dC_dmu_i.subs(values).evalf().tolist()
    print(f"Derivative w.r.t μ_{i} evaluated:", dC_dmu_num )
    dC_dmu.append(dC_dmu_num)

#### Implementação PyTorch:

In [ ]:
import torch

N = 3  # Number of components
mu = torch.tensor([[1.0+i, 1.0+i] for i in range(N)], requires_grad=True) 

p = torch.tensor([2.0, 3.0], requires_grad=False)
c = torch.tensor([1.0/(i+1) for i in range(N)], requires_grad=False)
Sigma_inv_i = torch.tensor([[1.0, 0.0],[0.0, 1.0]], requires_grad=False)
Sigma_inv_j = torch.tensor([[1.0, 0.0],[0.0, 1.0]], requires_grad=False)

# Forward pass
C = torch.tensor(0.0) 
for i in range(0,N):
    d_i = p - mu[i]
    G_i = torch.exp(-0.5 * (d_i.T @ Sigma_inv_i @ d_i))
    prod = 1
    for j in range(0, i):
        d_j = p - mu[j]
        G_j = torch.exp(-0.5 * (d_j.T @ Sigma_inv_j @ d_j))
        prod *= (1 - G_j)
    C += c[i] * G_i * prod
print("C(p)=", C)
# Backward pass
C.backward()

# Gradients
print(mu.grad)

#### Comparação:

In [ ]:
dC_dmu = torch.tensor(dC_dmu, dtype=torch.float32).reshape(mu.grad.shape)

# Gradients
print(mu.grad, "=\n", dC_dmu)

# Exemplo 11

$\displaystyle C(\mathbf{p}) = \sum_{i=1}^{N} \left[ c_i \ G_i(\mathbf{p})  \prod_{j=1}^{i-1} (1- G_j(\mathbf{p})) \right]$

Onde:
- $G_i(\mathbf{p}) = exp(-0.5 (\mathbf{p}-\mathbf{\mu_i})^T \Sigma_i^{'-1} (\mathbf{p}-\mathbf{\mu_i}))$;
- $\Sigma_i' = JWR_iS_iS_i^TR_i^TW^TJ^T$;
- $S_i = diag(\mathbf{s_i})$;
- $J \in \mathbb{R}^{2 \times 3}$ e $W,R_i,S_i \in \mathbb{R}^{3 \times 3}$;
- $c_i \in \mathbb{R}$, $\mathbf{\mu_i},\mathbf{p} \in \mathbb{R}^2$ e $\mathbf{s_i} \in \mathbb{R}^3$;
- $\mathbf{\mu_i}$ e $\mathbf{s_i}$ serão atualizados pelo gradiente descendente.

$\displaystyle \nabla(\mu) = \frac{\partial C}{\partial \mu} = \sum_{i=1}^{N} \left[ c_i \ \frac{\partial G_i}{\partial \mu} \prod_{j=1}^{i-1} (1- G_j)  + c_i G_i \sum_{k=1}^{i-1} \left(\prod_{j=1}^{k-1}  (1- G_j)\right) \left(-\frac{\partial G_k}{\partial \mu}\right) \left(\prod_{j=k+1}^{i-1}  (1- G_j)\right) \right]$

$\displaystyle \nabla(\mathbf{s}) = \frac{\partial C}{\partial \mathbf{s}} = \sum_{i=1}^{N} \left[ c_i \ \frac{\partial G_i}{\partial \mathbf{s}} \prod_{j=1}^{i-1} (1- G_j)  + c_i G_i \sum_{k=1}^{i-1} \left(\prod_{j=1}^{k-1}  (1- G_j)\right) \left(-\frac{\partial G_k}{\partial \mathbf{s}}\right) \left(\prod_{j=k+1}^{i-1}  (1- G_j)\right) \right]$

Para $N=3, \mathbf{p}=[2,3],\ c_i=1/(i+1),\ \mathbf{\mu_i}=[1+i,1+i],\ \mathbf{s_i} = [0.5(i+1),0.5(i+1),0.5(i+1)]$, $J = diag(1,1,0), W = I, R_i=I \rightarrow$

$\nabla(\mu) = ?$

$\nabla(\mathbf{s}) = ?$

#### Implementação SymPy:

In [ ]:
import sympy as sp

N = 3  # Number of components
R_i = sp.Matrix([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0],[0.0, 0.0, 1.0]]) # assumir conhecida por simplicidade
W = sp.Matrix([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0],[0.0, 0.0, 1.0]])
J = sp.Matrix([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0]])

# variam para n=1,...,N
c = sp.IndexedBase('c')
mux = sp.IndexedBase('μ_x')
muy = sp.IndexedBase('μ_y')
sx = sp.IndexedBase('s_x')
sy = sp.IndexedBase('s_y')
sz = sp.IndexedBase('s_z')

# Ponto de pesquisa
px, py= sp.symbols('p_x p_y')
p = sp.Matrix([px, py])

# Generate values to apply to the derivative
values = {px:2, py:3}
for i in range(N):
    values = values | {mux[i]:1+i, muy[i]:1+i, c[i]:1/(i+1), sx[i]:0.5*(i+1), sy[i]:0.5*(i+1), sz[i]:0.5*(i+1)}

# Expression for C(x)
C = 0 
for i in range(0,N):
    d_i = p - sp.Matrix([mux[i], muy[i]])
    S_i = sp.Matrix([[sx[i], 0, 0], [0, sy[i], 0], [0, 0, sz[i]]])
    Sigma_2D_i = J * W * R_i * S_i * S_i.T * R_i.T * W.T * J.T
    Sigma_2D_inv_i = Sigma_2D_i.inv()
    G_i = sp.exp(-0.5 * (d_i.T * Sigma_2D_inv_i * d_i)[0, 0])
    prod = 1
    for j in range(0, i):
        d_j = p - sp.Matrix([mux[j], muy[j]])
        S_j = sp.Matrix([[sx[j], 0, 0], [0, sy[j], 0], [0, 0, sz[j]]])
        Sigma_2D_j = J * W * R_i * S_j * S_j.T * R_i.T * W.T * J.T
        Sigma_2D_inv_j = Sigma_2D_j.inv()
        G_j = sp.exp(-0.5 * (d_j.T * Sigma_2D_inv_j * d_j)[0, 0])
        prod *= (1 - G_j)
    C += c[i] * G_i * prod

print("C(p)=")
display(C)

print(f"Evaluation data: {values}:")
print(f"C(p) evaluated:", C.subs(values).evalf())

dC_dmu = []
for i in range(N):
    dC_dmu_i = sp.diff(C, sp.Matrix([mux[i], muy[i]]))
    dC_dmu_num = dC_dmu_i.subs(values).evalf().tolist()
    print(f"Derivative w.r.t μ_{i} evaluated:", dC_dmu_num )
    dC_dmu.append(dC_dmu_num)

dC_ds = []
for i in range(N):
    dC_ds_i = sp.diff(C, sp.Matrix([sx[i], sy[i], sz[i]]))
    dC_ds_num = dC_ds_i.subs(values).evalf().tolist()
    print(f"Derivative w.r.t s_{i} evaluated:", dC_ds_num )
    dC_ds.append(dC_ds_num)

#### Implementação PyTorch:

In [ ]:
import torch

N = 3  # Number of components
mu = torch.tensor([[1.0+i, 1.0+i] for i in range(N)], requires_grad=True)
s = torch.tensor([[0.5*(i+1), 0.5*(i+1), 0.5*(i+1)] for i in range(N)], requires_grad=True) 

p = torch.tensor([2.0, 3.0], requires_grad=False)
c = torch.tensor([1.0/(i+1) for i in range(N)], requires_grad=False)
R_i = torch.tensor([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0],[0.0, 0.0, 1.0]], requires_grad=False)
W = torch.tensor([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0],[0.0, 0.0, 1.0]], requires_grad=False)
J = torch.tensor([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0]], requires_grad=False)

# Forward pass
C = torch.tensor(0.0) 
for i in range(N):
    d_i = p - mu[i]
    S_i = torch.diag(s[i])
    Sigma_2D_i = J @ W @ R_i @ S_i @ S_i.T @ R_i.T @ W.T @ J.T
    Sigma_2D_inv_i = torch.linalg.inv(Sigma_2D_i)
    G_i = torch.exp(-0.5 * (d_i.T @ Sigma_2D_inv_i @ d_i))
    prod = 1
    for j in range(i):
        d_j = p - mu[j]
        S_j = torch.diag(s[j])
        Sigma_2D_j = J @ W @ R_i @ S_j @ S_j.T @ R_i.T @ W.T @ J.T
        Sigma_2D_inv_j = torch.linalg.inv(Sigma_2D_j)
        G_j = torch.exp(-0.5 * (d_j.T @ Sigma_2D_inv_j @ d_j))
        prod *= (1 - G_j)
    C += c[i] * G_i * prod
print("C(p) =", C)

# Backward pass
C.backward()

# Gradients
print(mu.grad)
print(s.grad)

#### Comparação:

In [ ]:
dC_dmu = torch.tensor(dC_dmu, dtype=torch.float32).reshape(mu.grad.shape)
dC_ds = torch.tensor(dC_ds, dtype=torch.float32).reshape(s.grad.shape)

# Gradients
print(mu.grad, "=\n", dC_dmu, "\n")
print(s.grad, "=\n", dC_ds, "\n" )

# Exemplo 12

- $\displaystyle C(\mathbf{p_k}) = \sum_{i=1}^{N} \left[ c_i \ G_i(\mathbf{p_k})  \prod_{j=1}^{i-1} (1- G_j(\mathbf{p_k})) \right]$;

Onde:
- $G_i(\mathbf{p_k}) = exp(-0.5 (\mathbf{p_k}-\mathbf{\mu}^{2D}_i)^T \Sigma_i^{'-1} (\mathbf{p_k}-\mathbf{\mu}^{2D}_i))$;
- $\mu^{2D}_{i} = \begin{bmatrix}\frac{μ^{3D}_{ix} {P}_{0,0} + μ^{3D}_{iy} {P}_{0,1} + μ^{3D}_{iz} {P}_{0,2} + {P}_{0,3}}{μ^{3D}_{ix} {P}_{2,0} + μ^{3D}_{iy} {P}_{2,1} + μ^{3D}_{iz} {P}_{2,2} + {P}_{2,3}}\\\frac{μ^{3D}_{ix} {P}_{1,0} + μ^{3D}_{iy} {P}_{1,1} + μ^{3D}_{iz} {P}_{1,2} + {P}_{1,3}}{μ^{3D}_{ix} {P}_{2,0} + μ^{3D}_{iy} {P}_{2,1} + μ^{3D}_{iz} {P}_{2,2} + {P}_{2,3}}\end{bmatrix}$
- $\Sigma_i' = JWR_iS_iS_i^TR_i^TW^TJ^T$;
- $S_i = diag(\mathbf{s}_{i}) = \begin{bmatrix} s_{ix} & 0 & 0 \\ 0 & s_{iy} &0 \\ 0 & 0 & s_{iz} \end{bmatrix}$;
- $J \in \mathbb{R}^{2 \times 3}$ e $W,R_i,S_i \in \mathbb{R}^{3 \times 3}$;
- $c_i, P_{m,n} \in \mathbb{R}$, $\mathbf{\mu}^{2D}_i,\mathbf{p_k} \in \mathbb{R}^2$ e $\mathbf{\mu}^{3D}_i, \mathbf{s}_{i} \in \mathbb{R}^3$;
- $\mathbf{\mu}^{3D}_{i}$ e $\mathbf{s}_{i}$ serão atualizados pelo gradiente descendente.

$\displaystyle \nabla(μ^{3D}) = \frac{\partial C}{\partial μ^{3D}} = \sum_{i=1}^{N} \left[ c_i \ \frac{\partial G_i}{\partial μ^{3D}} \prod_{j=1}^{i-1} (1- G_j)  + c_i G_i \sum_{k=1}^{i-1} \left(\prod_{j=1}^{k-1}  (1- G_j)\right) \left(-\frac{\partial G_k}{\partial μ^{3D}}\right) \left(\prod_{j=k+1}^{i-1}  (1- G_j)\right) \right]$

$\displaystyle \nabla(\mathbf{s}) = \frac{\partial C}{\partial \mathbf{s}} = \sum_{i=1}^{N} \left[ c_i \ \frac{\partial G_i}{\partial \mathbf{s}} \prod_{j=1}^{i-1} (1- G_j)  + c_i G_i \sum_{k=1}^{i-1} \left(\prod_{j=1}^{k-1}  (1- G_j)\right) \left(-\frac{\partial G_k}{\partial \mathbf{s}}\right) \left(\prod_{j=k+1}^{i-1}  (1- G_j)\right) \right]$

Para $N=3, \mathbf{p}=[2,3],\ c_i=1/(i+1),\ \mathbf{\mu^{3D}_i}=[1+i,1+i,1+i],\ \mathbf{s_i} = [0.5(i+1),0.5(i+1),0.5(i+1)]$, $J = diag(1,1,0), P = diag(1,1,1,0), W = I, R_i=I \rightarrow$

$\nabla(\mu) = ?$

$\nabla(\mathbf{s}) = ?$

#### Implementação SymPy:

In [ ]:
import sympy as sp

N = 3  # Number of components
R_i = sp.Matrix([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0],[0.0, 0.0, 1.0]]) # assumir conhecida por simplicidade
W = sp.Matrix([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0],[0.0, 0.0, 1.0]])
J = sp.Matrix([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0]])
P = sp.Matrix([[1000.0, 0.0, 500.0, 300.0], [0.0, 1000.0, 500.0, 300.0], [300.0, 300.0, 300.0, 300.0]])

# variam para n=1,...,N
c = sp.IndexedBase('c')

mu3Dx = sp.IndexedBase('μ_x')
mu3Dy = sp.IndexedBase('μ_y')
mu3Dz = sp.IndexedBase('μ_z')

sx = sp.IndexedBase('s_x')
sy = sp.IndexedBase('s_y')
sz = sp.IndexedBase('s_z')

# Coordenada do ponto de pesquisa
p_k = sp.Matrix([[i, j] for i in range(16) for j in range(16)])

# Generate values to apply to the derivative
values = {}
for i in range(N):
    values = values | {mu3Dx[i]: 1+i, mu3Dy[i]: 1+i, mu3Dz[i]: 1+i,
                       sx[i]:0.5*(i+1), sy[i]:0.5*(i+1), sz[i]:0.5*(i+1),
                       c[i]:1/(i+1)}
def C_(p):
    # Expression for C(x)
    C_values = 0 
    for i in range(0,N):
        mu2Dx_i = (mu3Dx[i] * P[0, 0] + mu3Dy[i] * P[0, 1] + mu3Dz[i] * P[0, 2] + P[0, 3])/(mu3Dx[i] * P[2, 0] + mu3Dy[i] * P[2,1] + mu3Dz[i] * P[2, 2] + P[2, 3])
        mu2Dy_i = (mu3Dx[i] * P[1, 0] + mu3Dy[i] * P[1, 1] + mu3Dz[i] * P[1, 2] + P[1, 3])/(mu3Dx[i] * P[2, 0] + mu3Dy[i] * P[2,1] + mu3Dz[i] * P[2, 2] + P[2, 3])
        d_i = p - sp.Matrix([mu2Dx_i, mu2Dy_i])
        S_i = sp.Matrix([[sx[i], 0, 0], [0, sy[i], 0], [0, 0, sz[i]]])
        Sigma_2D_i = J * W * R_i * S_i * S_i.T * R_i.T * W.T * J.T
        Sigma_2D_inv_i = Sigma_2D_i.inv()
        G_i = sp.exp(-0.5 * (d_i.T * Sigma_2D_inv_i * d_i)[0, 0])
        prod = 1
        for j in range(0, i):
            mu2Dx_j = (mu3Dx[j] * P[0, 0] + mu3Dy[j] * P[0, 1] + mu3Dz[j] * P[0, 2] + P[0, 3])/(mu3Dx[j] * P[2, 0] + mu3Dy[j] * P[2,1] + mu3Dz[j] * P[2, 2] + P[2, 3])
            mu2Dy_j = (mu3Dx[j] * P[1, 0] + mu3Dy[j] * P[1, 1] + mu3Dz[j] * P[1, 2] + P[1, 3])/(mu3Dx[j] * P[2, 0] + mu3Dy[j] * P[2,1] + mu3Dz[j] * P[2, 2] + P[2, 3])
            d_j = p - sp.Matrix([mu2Dx_j, mu2Dy_j])
            S_j = sp.Matrix([[sx[j], 0, 0], [0, sy[j], 0], [0, 0, sz[j]]])
            Sigma_2D_j = J * W * R_i * S_j * S_j.T * R_i.T * W.T * J.T
            Sigma_2D_inv_j = Sigma_2D_j.inv()
            G_j = sp.exp(-0.5 * (d_j.T * Sigma_2D_inv_j * d_j)[0, 0])
            prod *= (1 - G_j)
        C_values += c[i] * G_i * prod
    return C_values

C = C_(p_k.row(16*2+3).T)

print("C(p)=")
display(C)

print(f"Evaluation data: {values}:")
print(f"C evaluated:", C.subs(values).evalf())

dC_dmu = []
for i in range(N):
    dC_dmu_i = sp.diff(C, sp.Matrix([mu3Dx[i], mu3Dy[i], mu3Dz[i]]))
    dC_dmu_num = dC_dmu_i.subs(values).evalf().tolist()
    print(f"Derivative w.r.t μ_{i} evaluated:", dC_dmu_num )
    dC_dmu.append(dC_dmu_num)

dC_ds = []
for i in range(N):
    dC_ds_i = sp.diff(C, sp.Matrix([sx[i], sy[i], sz[i]]))
    dC_ds_num = dC_ds_i.subs(values).evalf().tolist()
    print(f"Derivative w.r.t s_{i} evaluated:", dC_ds_num )
    dC_ds.append(dC_ds_num)


#### Implementação PyTorch:

In [ ]:
import torch

N = 3  # Number of components
mu3D = torch.tensor([[1.0+i, 1.0+i, 1.0+i] for i in range(N)], requires_grad=True)
s = torch.tensor([[0.5*(i+1), 0.5*(i+1), 0.5*(i+1)] for i in range(N)], requires_grad=True) 
p_k = torch.tensor([[i, j] for i in range(16) for j in range(16)], requires_grad=False)
c = torch.tensor([1.0/(i+1) for i in range(N)], requires_grad=False)

R_i = torch.tensor([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0],[0.0, 0.0, 1.0]], requires_grad=False)
W = torch.tensor([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0],[0.0, 0.0, 1.0]], requires_grad=False)
J = torch.tensor([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0]], requires_grad=False)
P = torch.tensor([[1000.0, 0.0, 500.0, 300.0], [0.0, 1000.0, 500.0, 300.0], [300.0, 300.0, 300.0, 300.0]], requires_grad=False)

#Forward pass
def C_(p):
    # Expression for C(x)
    C_values = torch.tensor(0.0) 
    for i in range(N):
        mu2Dx_i = (mu3D[i,0] * P[0, 0] + mu3D[i,1] * P[0, 1] + mu3D[i,2] * P[0, 2] + P[0, 3])/(mu3D[i,0] * P[2, 0] + mu3D[i,1] * P[2,1] + mu3D[i,2] * P[2, 2] + P[2, 3])
        mu2Dy_i = (mu3D[i,0] * P[1, 0] + mu3D[i,1] * P[1, 1] + mu3D[i,2] * P[1, 2] + P[1, 3])/(mu3D[i,0] * P[2, 0] + mu3D[i,1] * P[2,1] + mu3D[i,2] * P[2, 2] + P[2, 3])
        d_i = p - torch.stack([mu2Dx_i, mu2Dy_i])
        S_i = torch.diag(s[i])
        Sigma_2D_i = J @ W @ R_i @ S_i @ S_i.T @ R_i.T @ W.T @ J.T
        Sigma_2D_inv_i = torch.linalg.inv(Sigma_2D_i)
        G_i = torch.exp(-0.5 * (d_i.T @ Sigma_2D_inv_i @ d_i))
        prod = 1
        for j in range(i):
            mu2Dx_j = (mu3D[j,0] * P[0, 0] + mu3D[j,1] * P[0, 1] + mu3D[j,2] * P[0, 2] + P[0, 3])/(mu3D[j,0] * P[2, 0] + mu3D[j,1] * P[2,1] + mu3D[j,2] * P[2, 2] + P[2, 3])
            mu2Dy_j = (mu3D[j,0] * P[1, 0] + mu3D[j,1] * P[1, 1] + mu3D[j,2] * P[1, 2] + P[1, 3])/(mu3D[j,0] * P[2, 0] + mu3D[j,1] * P[2,1] + mu3D[j,2] * P[2, 2] + P[2, 3])
            d_j = p - torch.stack([mu2Dx_j, mu2Dy_j])
            S_j = torch.diag(s[j])
            Sigma_2D_j = J @ W @ R_i @ S_j @ S_j.T @ R_i.T @ W.T @ J.T
            Sigma_2D_inv_j = torch.linalg.inv(Sigma_2D_j)
            G_j = torch.exp(-0.5 * (d_j.T @ Sigma_2D_inv_j @ d_j))
            prod *= (1 - G_j)
        C_values += c[i] * G_i * prod
    return C_values
C = C_(p_k[16*2+3])
print("C(p) =", C)

# Backward pass
C.backward()

# Gradients
print(mu3D.grad)
print(s.grad)

#### Comparação:

In [ ]:
dC_dmu = torch.tensor(dC_dmu, dtype=torch.float32).reshape(mu3D.grad.shape)
dC_ds = torch.tensor(dC_ds, dtype=torch.float32).reshape(s.grad.shape)

# Gradients
print(mu3D.grad, "=\n", dC_dmu, "\n")
print(s.grad, "=\n", dC_ds, "\n" )

# Exemplo 13

- $\displaystyle C(\mathbf{p_k}) = \sum_{i=1}^{N} \left[ c_i \ G_i(\mathbf{p_k})  \prod_{j=1}^{i-1} (1- G_j(\mathbf{p_k})) \right]$;

Onde:
- $\displaystyle c_i = \sum_{l=0}^{l_{max}}\sum_{m=-l}^{l}k_l^mY_l^m$;
- $Y_l^m$ é uma função harmônica esférica de grau $l$ e ordem $m$;
- $G_i(\mathbf{p_k}) = exp(-0.5 (\mathbf{p_k}-\mathbf{\mu}^{2D}_i)^T \Sigma_i^{'-1} (\mathbf{p_k}-\mathbf{\mu}^{2D}_i))$;

- $\mu^{2D}_{i} = \begin{bmatrix}\frac{μ^{3D}_{ix} {P}_{0,0} + μ^{3D}_{iy} {P}_{0,1} + μ^{3D}_{iz} {P}_{0,2} + {P}_{0,3}}{μ^{3D}_{ix} {P}_{2,0} + μ^{3D}_{iy} {P}_{2,1} + μ^{3D}_{iz} {P}_{2,2} + {P}_{2,3}}\\\frac{μ^{3D}_{ix} {P}_{1,0} + μ^{3D}_{iy} {P}_{1,1} + μ^{3D}_{iz} {P}_{1,2} + {P}_{1,3}}{μ^{3D}_{ix} {P}_{2,0} + μ^{3D}_{iy} {P}_{2,1} + μ^{3D}_{iz} {P}_{2,2} + {P}_{2,3}}\end{bmatrix}$ , $P_{m,n}$ são as entradas de uma matriz $P \in \mathbb{R}^{3 \times 4}$;
- $\Sigma_i' = JWR_iS_iS_i^TR_i^TW^TJ^T$;
- $S_i = diag(\mathbf{s}_{i}) = \begin{bmatrix} s_{ix} & 0 & 0 \\ 0 & s_{iy} &0 \\ 0 & 0 & s_{iz} \end{bmatrix}$;
- $J \in \mathbb{R}^{2 \times 3}$ e $W,R_i \in \mathbb{R}^{3 \times 3}$;
- ${k_{li}^m} \in \mathbb{R}$, $\mathbf{\mu}^{2D}_i,\mathbf{p_k} \in \mathbb{R}^2$ e $cp, \mathbf{\mu}^{3D}_i, \mathbf{s}_{i} \in \mathbb{R}^3$;
- $\mathbf{\mu}^{3D}_{i}, {k_{li}^m}$ e $\mathbf{s}_{i}$ serão atualizados pelo gradiente descendente.

$\displaystyle \nabla(μ^{3D}) = \frac{\partial C}{\partial μ^{3D}} = \sum_{i=1}^{N} \left[ c_i \ \frac{\partial G_i}{\partial μ^{3D}} \prod_{j=1}^{i-1} (1- G_j)  + c_i G_i \sum_{k=1}^{i-1} \left(\prod_{j=1}^{k-1}  (1- G_j)\right) \left(-\frac{\partial G_k}{\partial μ^{3D}}\right) \left(\prod_{j=k+1}^{i-1}  (1- G_j)\right) \right]$

$\displaystyle \nabla(\mathbf{s}) = \frac{\partial C}{\partial \mathbf{s}} = \sum_{i=1}^{N} \left[ c_i \ \frac{\partial G_i}{\partial \mathbf{s}} \prod_{j=1}^{i-1} (1- G_j)  + c_i G_i \sum_{k=1}^{i-1} \left(\prod_{j=1}^{k-1}  (1- G_j)\right) \left(-\frac{\partial G_k}{\partial \mathbf{s}}\right) \left(\prod_{j=k+1}^{i-1}  (1- G_j)\right) \right]$

$\displaystyle \nabla(k) = \frac{\partial C}{\partial k} = \sum_{i=1}^{N} \left[ \frac{\partial c_i}{\partial k} \ G_i \prod_{j=1}^{i-1} (1- G_j) \right]$

Para $N=3, l_{max}=1, k_{li}^m=1/(i+1), cp = [0,0,0],  \mathbf{p}_k=[2,3],\ \mathbf{\mu^{3D}_i}=[1+i,1+i,1+i],\ \mathbf{s_i} = [0.5(i+1),0.5(i+1),0.5(i+1)]$, $J = diag(1,1,0), P = diag(1,1,1,0), W = I, R_i=I \rightarrow$

$\nabla(\mu) = ?$

$\nabla(\mathbf{s}) = ?$

$\nabla(k) = ?$

#### Implementação SymPy:

In [ ]:
import sympy as sp

def build_color(sh, rays_d):
    lmax = 1  # Maximum spherical harmonics degree
    C0 = 0.28209479177387814
    C1 = 0.4886025119029199

    color = C0 * sh[:, 0][0]
    if lmax > 0:
        x, y, z = rays_d[0, :][0], rays_d[1, :][0], rays_d[2, :][0]
        color = (color -
                C1 * y * sh[:, 1][0] +
                C1 * z * sh[:, 2][0] -
                C1 * x * sh[:, 3][0])
    
    color = (color + 0.5)
    return color

N = 3  # Number of components
R_i = sp.Matrix([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0],[0.0, 0.0, 1.0]]) # assumir conhecida por simplicidade
W = sp.Matrix([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0],[0.0, 0.0, 1.0]])
J = sp.Matrix([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0]])
P = sp.Matrix([[1000.0, 0.0, 500.0, 300.0], [0.0, 1000.0, 500.0, 300.0], [300.0, 300.0, 300.0, 300.0]])
cp = sp.Matrix([0.0, 0.0, 0.0])


# variam para n=1,...,N
sh0 = sp.IndexedBase('sh_0') # k_0^0   <-- grau_max = 0 
sh1 = sp.IndexedBase('sh_1') # k_1^{-1}
sh2 = sp.IndexedBase('sh_2') # k_1^{0}
sh3 = sp.IndexedBase('sh_3') # k_1^{1} <-- grau_max = 1 

mu3Dx = sp.IndexedBase('μ_x')
mu3Dy = sp.IndexedBase('μ_y')
mu3Dz = sp.IndexedBase('μ_z')

sx = sp.IndexedBase('s_x')
sy = sp.IndexedBase('s_y')
sz = sp.IndexedBase('s_z')

# Coordenada do ponto de pesquisa
p_k = sp.Matrix([[i, j] for i in range(16) for j in range(16)])

# Generate values to apply to the derivative
values = {}
for i in range(N):
    values = values | {mu3Dx[i]: 1+i, mu3Dy[i]: 1+i, mu3Dz[i]: 1+i,
                       sx[i]:0.5*(i+1), sy[i]:0.5*(i+1), sz[i]:0.5*(i+1),
                       sh0[i]:1/(i+1),sh1[i]:1/(i+1), sh2[i]:1/(i+1), sh3[i]:1/(i+1) }
def C_(p):
    # Expression for C(x)
    C_values = 0 
    for i in range(0,N):   
        sh_i = sp.Matrix([sh0[i], sh1[i], sh2[i], sh3[i]])
        mu3D_i = sp.Matrix([mu3Dx[i], mu3Dy[i], mu3Dz[i]])
        mu2Dx_i = (mu3Dx[i] * P[0, 0] + mu3Dy[i] * P[0, 1] + mu3Dz[i] * P[0, 2] + P[0, 3])/(mu3Dx[i] * P[2, 0] + mu3Dy[i] * P[2,1] + mu3Dz[i] * P[2, 2] + P[2, 3])
        mu2Dy_i = (mu3Dx[i] * P[1, 0] + mu3Dy[i] * P[1, 1] + mu3Dz[i] * P[1, 2] + P[1, 3])/(mu3Dx[i] * P[2, 0] + mu3Dy[i] * P[2,1] + mu3Dz[i] * P[2, 2] + P[2, 3])
        d_i = p - sp.Matrix([mu2Dx_i, mu2Dy_i])
        S_i = sp.Matrix([[sx[i], 0, 0], [0, sy[i], 0], [0, 0, sz[i]]])
        Sigma_2D_i = J * W * R_i * S_i * S_i.T * R_i.T * W.T * J.T
        Sigma_2D_inv_i = Sigma_2D_i.inv()
        G_i = sp.exp(-0.5 * (d_i.T * Sigma_2D_inv_i * d_i)[0, 0])
        prod = 1
        for j in range(0, i):
            mu2Dx_j = (mu3Dx[j] * P[0, 0] + mu3Dy[j] * P[0, 1] + mu3Dz[j] * P[0, 2] + P[0, 3])/(mu3Dx[j] * P[2, 0] + mu3Dy[j] * P[2,1] + mu3Dz[j] * P[2, 2] + P[2, 3])
            mu2Dy_j = (mu3Dx[j] * P[1, 0] + mu3Dy[j] * P[1, 1] + mu3Dz[j] * P[1, 2] + P[1, 3])/(mu3Dx[j] * P[2, 0] + mu3Dy[j] * P[2,1] + mu3Dz[j] * P[2, 2] + P[2, 3])
            d_j = p - sp.Matrix([mu2Dx_j, mu2Dy_j])
            S_j = sp.Matrix([[sx[j], 0, 0], [0, sy[j], 0], [0, 0, sz[j]]])
            Sigma_2D_j = J * W * R_i * S_j * S_j.T * R_i.T * W.T * J.T
            Sigma_2D_inv_j = Sigma_2D_j.inv()
            G_j = sp.exp(-0.5 * (d_j.T * Sigma_2D_inv_j * d_j)[0, 0])
            prod *= (1 - G_j)
        direction = mu3D_i - cp
        c_i = build_color(sh_i.T, direction)
        C_values += c_i * G_i * prod
    return C_values

C = C_(p_k.row(16*2+3).T)

print("C(p)=")
display(C)

print(f"Evaluation data: {values}:")
print(f"C evaluated:", C.subs(values).evalf())

dC_dmu = []
for i in range(N):
    dC_dmu_i = sp.diff(C, sp.Matrix([mu3Dx[i], mu3Dy[i], mu3Dz[i]]))
    dC_dmu_num = dC_dmu_i.subs(values).evalf().tolist()
    print(f"Derivative w.r.t μ_{i} evaluated:", dC_dmu_num )
    dC_dmu.append(dC_dmu_num)

dC_ds = []
for i in range(N):
    dC_ds_i = sp.diff(C, sp.Matrix([sx[i], sy[i], sz[i]]))
    dC_ds_num = dC_ds_i.subs(values).evalf().tolist()
    print(f"Derivative w.r.t s_{i} evaluated:", dC_ds_num )
    dC_ds.append(dC_ds_num)

dC_dsh = []
for i in range(N):
    dC_dsh_i = sp.diff(C, sp.Matrix([sh0[i], sh1[i], sh2[i], sh3[i]]))
    dC_dsh_num = dC_dsh_i.subs(values).evalf().tolist()
    print(f"Derivative w.r.t sh_{i} evaluated:",dC_dsh_num )
    dC_dsh.append(dC_dsh_num)


#### Implementação PyTorch:

In [ ]:
import torch

def build_color(sh_i, rays_d):
    lmax = 1  # Maximum spherical harmonics degree
    C0 = 0.28209479177387814
    C1 = 0.4886025119029199
    color = C0 * sh_i[..., 0]
    if lmax > 0:
        x, y, z = rays_d[0, ...], rays_d[1, ...], rays_d[2, ...]
        color = (color -
                C1 * y * sh_i[..., 1] +
                C1 * z * sh_i[..., 2] -
                C1 * x * sh_i[..., 3])
    
    color = (color + 0.5)
    return color

N = 3  # Number of components
mu3D = torch.tensor([[1.0+i, 1.0+i, 1.0+i] for i in range(N)], requires_grad=True)
s = torch.tensor([[0.5*(i+1), 0.5*(i+1), 0.5*(i+1)] for i in range(N)], requires_grad=True) 
sh = torch.tensor([[1.0/(i+1), 1.0/(i+1), 1.0/(i+1), 1.0/(i+1)] for i in range(N)], requires_grad=True)

p_k = torch.tensor([[i, j] for i in range(16) for j in range(16)], requires_grad=False)
c = torch.tensor([1.0/(i+1) for i in range(N)], requires_grad=False)
R_i = torch.tensor([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0],[0.0, 0.0, 1.0]], requires_grad=False)
W = torch.tensor([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0],[0.0, 0.0, 1.0]], requires_grad=False)
J = torch.tensor([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0]], requires_grad=False)
P = torch.tensor([[1000.0, 0.0, 500.0, 300.0], [0.0, 1000.0, 500.0, 300.0], [300.0, 300.0, 300.0, 300.0]], requires_grad=False)
cp = torch.tensor([0.0, 0.0, 0.0], requires_grad=False)

#Forward pass
def C_(p):
    # Expression for C(x)
    C_values = torch.tensor(0.0) 
    for i in range(N):
        mu2Dx_i = (mu3D[i,0] * P[0, 0] + mu3D[i,1] * P[0, 1] + mu3D[i,2] * P[0, 2] + P[0, 3])/(mu3D[i,0] * P[2, 0] + mu3D[i,1] * P[2,1] + mu3D[i,2] * P[2, 2] + P[2, 3])
        mu2Dy_i = (mu3D[i,0] * P[1, 0] + mu3D[i,1] * P[1, 1] + mu3D[i,2] * P[1, 2] + P[1, 3])/(mu3D[i,0] * P[2, 0] + mu3D[i,1] * P[2,1] + mu3D[i,2] * P[2, 2] + P[2, 3])
        d_i = p - torch.stack([mu2Dx_i, mu2Dy_i])
        S_i = torch.diag(s[i])
        Sigma_2D_i = J @ W @ R_i @ S_i @ S_i.T @ R_i.T @ W.T @ J.T
        Sigma_2D_inv_i = torch.linalg.inv(Sigma_2D_i)
        G_i = torch.exp(-0.5 * (d_i.T @ Sigma_2D_inv_i @ d_i))
        prod = 1
        for j in range(i):
            mu2Dx_j = (mu3D[j,0] * P[0, 0] + mu3D[j,1] * P[0, 1] + mu3D[j,2] * P[0, 2] + P[0, 3])/(mu3D[j,0] * P[2, 0] + mu3D[j,1] * P[2,1] + mu3D[j,2] * P[2, 2] + P[2, 3])
            mu2Dy_j = (mu3D[j,0] * P[1, 0] + mu3D[j,1] * P[1, 1] + mu3D[j,2] * P[1, 2] + P[1, 3])/(mu3D[j,0] * P[2, 0] + mu3D[j,1] * P[2,1] + mu3D[j,2] * P[2, 2] + P[2, 3])
            d_j = p - torch.stack([mu2Dx_j, mu2Dy_j])
            S_j = torch.diag(s[j])
            Sigma_2D_j = J @ W @ R_i @ S_j @ S_j.T @ R_i.T @ W.T @ J.T
            Sigma_2D_inv_j = torch.linalg.inv(Sigma_2D_j)
            G_j = torch.exp(-0.5 * (d_j.T @ Sigma_2D_inv_j @ d_j))
            prod *= (1 - G_j)
        direction = mu3D[i] - cp
        c_i = build_color(sh[i], direction)
        C_values += c_i * G_i * prod
    return C_values
C = C_(p_k[16*2+3])
print("C(p) =", C)

# Backward pass
C.backward()

# Gradients
print(mu3D.grad)
print(s.grad)
print(sh.grad)

#### Comparação:

In [ ]:
dC_dmu = torch.tensor(dC_dmu, dtype=torch.float32).reshape(mu3D.grad.shape)
dC_ds = torch.tensor(dC_ds, dtype=torch.float32).reshape(s.grad.shape)
dC_dsh = torch.tensor(dC_dsh, dtype=torch.float32).reshape(sh.grad.shape)

# Gradients
print(mu3D.grad, "=\n", dC_dmu, "\n")
print(s.grad, "=\n", dC_ds, "\n" )
print(sh.grad, "=\n", dC_dsh, "\n" )

# Exemplo 14

$\displaystyle \mathcal{L} = L_{recon}(C, C')$

Onde:
- $C$ e $C'$ são as imagens, reconstruída e prevista respectivamente, vetorizadas com $M$ pixels cada
- $\displaystyle L_{recon}(C, C') =\sum_{k=1}^{M} || C(\mathbf{p_k}) - C'(\mathbf{p_k}) ||^2 $
- $\displaystyle C(\mathbf{p_k}) = \sum_{i=1}^{N} \left[ c_i \ G_i(\mathbf{p_k})  \prod_{j=1}^{i-1} (1- G_j(\mathbf{p_k})) \right]$;
- $\displaystyle c_i = \sum_{l=0}^{l_{max}}\sum_{m=-l}^{l}k_l^mY_l^m$;
- $Y_l^m$ é uma função harmônica esférica de grau $l$ e ordem $m$;
- $G_i(\mathbf{p_k}) = exp(-0.5 (\mathbf{p_k}-\mathbf{\mu}^{2D}_i)^T \Sigma_i^{'-1} (\mathbf{p_k}-\mathbf{\mu}^{2D}_i))$;

- $\mu^{2D}_{i} = \begin{bmatrix}\frac{μ^{3D}_{ix} {P}_{0,0} + μ^{3D}_{iy} {P}_{0,1} + μ^{3D}_{iz} {P}_{0,2} + {P}_{0,3}}{μ^{3D}_{ix} {P}_{2,0} + μ^{3D}_{iy} {P}_{2,1} + μ^{3D}_{iz} {P}_{2,2} + {P}_{2,3}}\\\frac{μ^{3D}_{ix} {P}_{1,0} + μ^{3D}_{iy} {P}_{1,1} + μ^{3D}_{iz} {P}_{1,2} + {P}_{1,3}}{μ^{3D}_{ix} {P}_{2,0} + μ^{3D}_{iy} {P}_{2,1} + μ^{3D}_{iz} {P}_{2,2} + {P}_{2,3}}\end{bmatrix}$ , $P_{m,n}$ são as entradas de uma matriz $P \in \mathbb{R}^{3 \times 4}$;
- $\Sigma_i' = JWR_iS_iS_i^TR_i^TW^TJ^T$;
- $S_i = diag(\mathbf{s}_{i}) = \begin{bmatrix} s_{ix} & 0 & 0 \\ 0 & s_{iy} &0 \\ 0 & 0 & s_{iz} \end{bmatrix}$;
- $J \in \mathbb{R}^{2 \times 3}$ e $W,R_i \in \mathbb{R}^{3 \times 3}$;
- ${k_{li}^m} \in \mathbb{R}$, $\mathbf{\mu}^{2D}_i,\mathbf{p_k} \in \mathbb{R}^2$ e $cp, \mathbf{\mu}^{3D}_i, \mathbf{s}_{i} \in \mathbb{R}^3$;
- $\mathbf{\mu}^{3D}_{i}, {k_{li}^m}$ e $\mathbf{s}_{i}$ serão atualizados pelo gradiente descendente.

$\displaystyle \nabla(\mu^{3D}) = \frac{\partial \mathcal{L}}{\partial \mu^{3D}} = \sum_{k=1}^{256} 2 \cdot || C - C' || \cdot || \frac{\partial C}{\partial \mu^{3D}}|| $

$\displaystyle \nabla(\mathbf{s}) = \frac{\partial \mathcal{L}}{\partial \mathbf{s}} = \sum_{k=1}^{256} 2 \cdot || C - C' || \cdot || \frac{\partial C}{\partial \mathbf{s}}||$

$\displaystyle \nabla(k) = \frac{\partial \mathcal{L}}{\partial k} = \sum_{k=1}^{256} 2 \cdot || C - C' || \cdot || \frac{\partial C}{\partial k}||$

Para $N=3, M=4, C'(\mathbf{p_k}) = k, l_{max}=1, k_{li}^m=1/(i+1), cp = [0,0,0], \ \mathbf{\mu^{3D}_i}=[1+i,1+i,1+i],\ \mathbf{s_i} = [0.5(i+1),0.5(i+1),0.5(i+1)]$, $J = diag(1,1,0), P = diag(1,1,1,0), W = I, R_i=I \rightarrow$

$\nabla(\mu) = ?$

$\nabla(\mathbf{s}) = ?$

$\nabla(k) = ?$

#### Implementação SymPy:

In [ ]:
import sympy as sp
from tqdm.notebook import tqdm

def build_color(sh, rays_d):
    lmax = 1  # Maximum spherical harmonics degree
    C0 = 0.28209479177387814
    C1 = 0.4886025119029199

    color = C0 * sh[:, 0][0]
    if lmax > 0:
        x, y, z = rays_d[0, :][0], rays_d[1, :][0], rays_d[2, :][0]
        color = (color -
                C1 * y * sh[:, 1][0] +
                C1 * z * sh[:, 2][0] -
                C1 * x * sh[:, 3][0])
    
    color = (color + 0.5)
    return color

N = 3  # Number of gaussians
M = 4 # Number of pixels
R_i = sp.Matrix([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0],[0.0, 0.0, 1.0]]) # assumir conhecida por simplicidade
W = sp.Matrix([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0],[0.0, 0.0, 1.0]])
J = sp.Matrix([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0]])
P = sp.Matrix([[1000.0, 0.0, 500.0, 300.0], [0.0, 1000.0, 500.0, 300.0], [300.0, 300.0, 300.0, 300.0]])
cp = sp.Matrix([0.0, 0.0, 0.0])

# variam para n=1,...,N
sh0 = sp.IndexedBase('sh_0') # k_0^0   <-- grau_max = 0 
sh1 = sp.IndexedBase('sh_1') # k_1^{-1}
sh2 = sp.IndexedBase('sh_2') # k_1^{0}
sh3 = sp.IndexedBase('sh_3') # k_1^{1} <-- grau_max = 1 

mu3Dx = sp.IndexedBase('μ_x')
mu3Dy = sp.IndexedBase('μ_y')
mu3Dz = sp.IndexedBase('μ_z')

sx = sp.IndexedBase('s_x')
sy = sp.IndexedBase('s_y')
sz = sp.IndexedBase('s_z')

# Coordenada do ponto de pesquisa
p_k = sp.Matrix([[i, j] for i in range(int(M**(0.5))) for j in range(int(M**(0.5)))])

# Observações
C_hat = sp.Matrix([[k] for k in range(M)])

# Generate values to apply to the derivative
values = {}
for i in range(N):
    values = values | {mu3Dx[i]: 1+i, mu3Dy[i]: 1+i, mu3Dz[i]: 1+i,
                       sx[i]:0.5*(i+1), sy[i]:0.5*(i+1), sz[i]:0.5*(i+1),
                       sh0[i]:1/(i+1),sh1[i]:1/(i+1), sh2[i]:1/(i+1), sh3[i]:1/(i+1) }
def C_(p):
    # Expression for C(x)
    C_values = 0 
    for i in range(0,N):   
        sh_i = sp.Matrix([sh0[i], sh1[i], sh2[i], sh3[i]])
        mu3D_i = sp.Matrix([mu3Dx[i], mu3Dy[i], mu3Dz[i]])
        mu2Dx_i = (mu3Dx[i] * P[0, 0] + mu3Dy[i] * P[0, 1] + mu3Dz[i] * P[0, 2] + P[0, 3])/(mu3Dx[i] * P[2, 0] + mu3Dy[i] * P[2,1] + mu3Dz[i] * P[2, 2] + P[2, 3])
        mu2Dy_i = (mu3Dx[i] * P[1, 0] + mu3Dy[i] * P[1, 1] + mu3Dz[i] * P[1, 2] + P[1, 3])/(mu3Dx[i] * P[2, 0] + mu3Dy[i] * P[2,1] + mu3Dz[i] * P[2, 2] + P[2, 3])
        d_i = p - sp.Matrix([mu2Dx_i, mu2Dy_i])
        S_i = sp.Matrix([[sx[i], 0, 0], [0, sy[i], 0], [0, 0, sz[i]]])
        Sigma_2D_i = J * W * R_i * S_i * S_i.T * R_i.T * W.T * J.T
        Sigma_2D_inv_i = Sigma_2D_i.inv()
        G_i = sp.exp(-0.5 * (d_i.T * Sigma_2D_inv_i * d_i)[0, 0])
        prod = 1
        for j in range(0, i):
            mu2Dx_j = (mu3Dx[j] * P[0, 0] + mu3Dy[j] * P[0, 1] + mu3Dz[j] * P[0, 2] + P[0, 3])/(mu3Dx[j] * P[2, 0] + mu3Dy[j] * P[2,1] + mu3Dz[j] * P[2, 2] + P[2, 3])
            mu2Dy_j = (mu3Dx[j] * P[1, 0] + mu3Dy[j] * P[1, 1] + mu3Dz[j] * P[1, 2] + P[1, 3])/(mu3Dx[j] * P[2, 0] + mu3Dy[j] * P[2,1] + mu3Dz[j] * P[2, 2] + P[2, 3])
            d_j = p - sp.Matrix([mu2Dx_j, mu2Dy_j])
            S_j = sp.Matrix([[sx[j], 0, 0], [0, sy[j], 0], [0, 0, sz[j]]])
            Sigma_2D_j = J * W * R_i * S_j * S_j.T * R_i.T * W.T * J.T
            Sigma_2D_inv_j = Sigma_2D_j.inv()
            G_j = sp.exp(-0.5 * (d_j.T * Sigma_2D_inv_j * d_j)[0, 0])
            prod *= (1 - G_j)
        direction = mu3D_i - cp
        c_i = build_color(sh_i.T, direction)
        C_values += c_i * G_i * prod
    return C_values

L = 0
for k in tqdm(range(M)):
    L += (C_(p_k.row(k).T) - C_hat[k])**2

print("L=")
display(L)

print(f"Evaluation data: {values}:")
print(f"L evaluated:", L.subs(values).evalf())

### Sympy can't handle this size of derivatives...
dL_dmu = []
for i in tqdm(range(N)):
    dL_dmu_i = sp.diff(L, sp.Matrix([mu3Dx[i], mu3Dy[i], mu3Dz[i]]))
    dL_dmu_num = dL_dmu_i.subs(values).evalf().tolist()
    print(f"Derivative w.r.t μ_{i} evaluated:", dL_dmu_num )
    dL_dmu.append(dL_dmu_num)

dL_ds = []
for i in tqdm(range(N)):
    dL_ds_i = sp.diff(L, sp.Matrix([sx[i], sy[i], sz[i]]))
    dL_ds_num = dL_ds_i.subs(values).evalf().tolist()
    print(f"Derivative w.r.t s_{i} evaluated:", dL_ds_num )
    dL_ds.append(dL_ds_num)

dL_dsh = []
for i in tqdm(range(N)):
    dL_dsh_i = sp.diff(L, sp.Matrix([sh0[i], sh1[i], sh2[i], sh3[i]]))
    dL_dsh_num = dL_dsh_i.subs(values).evalf().tolist()
    print(f"Derivative w.r.t sh_{i} evaluated:",dL_dsh_num )
    dL_dsh.append(dL_dsh_num)


#### Implementação PyTorch:

In [ ]:
import torch

def build_color(sh_i, rays_d):
    lmax = 1  # Maximum spherical harmonics degree
    C0 = 0.28209479177387814
    C1 = 0.4886025119029199
    color = C0 * sh_i[..., 0]
    if lmax > 0:
        x, y, z = rays_d[0, ...], rays_d[1, ...], rays_d[2, ...]
        color = (color -
                C1 * y * sh_i[..., 1] +
                C1 * z * sh_i[..., 2] -
                C1 * x * sh_i[..., 3])
    
    color = (color + 0.5)
    return color

N = 3  # Number of gaussians
M = 4 # Number of pixels
mu3D = torch.tensor([[1.0+i, 1.0+i, 1.0+i] for i in range(N)], requires_grad=True)
s = torch.tensor([[0.5*(i+1), 0.5*(i+1), 0.5*(i+1)] for i in range(N)], requires_grad=True) 
sh = torch.tensor([[1.0/(i+1), 1.0/(i+1), 1.0/(i+1), 1.0/(i+1)] for i in range(N)], requires_grad=True)

p_k = torch.tensor([[i, j] for i in range(int(M**(0.5))) for j in range(int(M**(0.5)))], requires_grad=False)
C_hat = torch.tensor([k for k in range(M)])
c = torch.tensor([1.0/(i+1) for i in range(N)], requires_grad=False)
R_i = torch.tensor([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0],[0.0, 0.0, 1.0]], requires_grad=False)
W = torch.tensor([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0],[0.0, 0.0, 1.0]], requires_grad=False)
J = torch.tensor([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0]], requires_grad=False)
P = torch.tensor([[1000.0, 0.0, 500.0, 300.0], [0.0, 1000.0, 500.0, 300.0], [300.0, 300.0, 300.0, 300.0]], requires_grad=False)
cp = torch.tensor([0.0, 0.0, 0.0], requires_grad=False)

#Forward pass
def C_(p):
    # Expression for C(x)
    C_values = torch.tensor(0.0) 
    for i in range(N):
        mu2Dx_i = (mu3D[i,0] * P[0, 0] + mu3D[i,1] * P[0, 1] + mu3D[i,2] * P[0, 2] + P[0, 3])/(mu3D[i,0] * P[2, 0] + mu3D[i,1] * P[2,1] + mu3D[i,2] * P[2, 2] + P[2, 3])
        mu2Dy_i = (mu3D[i,0] * P[1, 0] + mu3D[i,1] * P[1, 1] + mu3D[i,2] * P[1, 2] + P[1, 3])/(mu3D[i,0] * P[2, 0] + mu3D[i,1] * P[2,1] + mu3D[i,2] * P[2, 2] + P[2, 3])
        d_i = p - torch.stack([mu2Dx_i, mu2Dy_i])
        S_i = torch.diag(s[i])
        Sigma_2D_i = J @ W @ R_i @ S_i @ S_i.T @ R_i.T @ W.T @ J.T
        Sigma_2D_inv_i = torch.linalg.inv(Sigma_2D_i)
        G_i = torch.exp(-0.5 * (d_i.T @ Sigma_2D_inv_i @ d_i))
        prod = 1
        for j in range(i):
            mu2Dx_j = (mu3D[j,0] * P[0, 0] + mu3D[j,1] * P[0, 1] + mu3D[j,2] * P[0, 2] + P[0, 3])/(mu3D[j,0] * P[2, 0] + mu3D[j,1] * P[2,1] + mu3D[j,2] * P[2, 2] + P[2, 3])
            mu2Dy_j = (mu3D[j,0] * P[1, 0] + mu3D[j,1] * P[1, 1] + mu3D[j,2] * P[1, 2] + P[1, 3])/(mu3D[j,0] * P[2, 0] + mu3D[j,1] * P[2,1] + mu3D[j,2] * P[2, 2] + P[2, 3])
            d_j = p - torch.stack([mu2Dx_j, mu2Dy_j])
            S_j = torch.diag(s[j])
            Sigma_2D_j = J @ W @ R_i @ S_j @ S_j.T @ R_i.T @ W.T @ J.T
            Sigma_2D_inv_j = torch.linalg.inv(Sigma_2D_j)
            G_j = torch.exp(-0.5 * (d_j.T @ Sigma_2D_inv_j @ d_j))
            prod *= (1 - G_j)
        direction = mu3D[i] - cp
        c_i = build_color(sh[i], direction)
        C_values += c_i * G_i * prod
    return C_values

L = torch.tensor(0.0) 
for k in tqdm(range(M)):
    C = C_(p_k[k])
    L += ( C - C_hat[k])**2

print("L=", L)

# Backward pass
L.backward()

# Gradients
print(mu3D.grad)
print(s.grad)
print(sh.grad)

#### Comparação

In [ ]:
dL_dmu = torch.tensor(dL_dmu, dtype=torch.float32).reshape(mu3D.grad.shape)
dL_ds = torch.tensor(dL_ds, dtype=torch.float32).reshape(s.grad.shape)
dL_dsh = torch.tensor(dL_dsh, dtype=torch.float32).reshape(sh.grad.shape)

# Gradients
print(mu3D.grad, "=\n", dL_dmu, "\n")
print(s.grad, "=\n", dL_ds, "\n" )
print(sh.grad, "=\n", dL_dsh, "\n" )

# Exemplo 15

$\displaystyle \mathcal{L}(C, C') = L_{DSSIM}(C, C')$

- $C$ e $C'$ são as imagens reconstruída e prevista respectivamente, vetorizadas com $M$ pixels cada

- $\displaystyle L_{DSSIM}(C, C') = 1- \left(\frac{2\overline{C}\ \overline{C'}+ 10^{-9}}{\overline{C}^2 + \overline{C'}^2 + 10^{-9}}\right) \cdot \left(\frac{2\sigma_{CC'}+ 10^{-9}}{\sigma_{C}^2 + \sigma_{C'}^2 + 10^{-9}}\right)$
- $\mathbf{p_k}$ é a coordenada do pixel de posição $k$ em cada imagem vetorizada

- $\overline{C}$, $\overline{C'}$ são as médias de $C$ e $C'$, ou seja:
    - $ \displaystyle \overline{C} = \frac{1}{M} \sum_{k=1}^{M}  C(\mathbf{p_k})$
    - $ \displaystyle \overline{C'} = \frac{1}{M} \sum_{k=1}^{M}  C'(\mathbf{p_k})$
- $\sigma_{C}$, $\sigma_{C'}$ são as variâncias de $C$ e $C'$, ou seja:
    - $ \displaystyle \sigma_{C} = \frac{1}{M} \sum_{k=1}^{M} (C(\mathbf{p_k}) - \overline{C} )^2  $
    - $ \displaystyle \sigma_{C'} = \frac{1}{M} \sum_{k=1}^{M} (C'(\mathbf{p_k}) - \overline{C'} )^2  $
- $\sigma_{CC'}$ é a covariância de $C$ e $C'$, ou seja:
    - $ \displaystyle \sigma_{CC'} = \frac{1}{M} \sum_{k=1}^{M} (C(\mathbf{p_k}) - \overline{C} )(C'(\mathbf{p_k}) - \overline{C'} )  $
- $\displaystyle C(\mathbf{p_k}) = \sum_{i=1}^{N} \left[ c_i \ G_i(\mathbf{p_k})  \prod_{j=1}^{i-1} (1- G_j(\mathbf{p_k})) \right]$;
- $\displaystyle c_i = \sum_{l=0}^{l_{max}}\sum_{m=-l}^{l}k_l^mY_l^m$;
- $Y_l^m$ é uma função harmônica esférica de grau $l$ e ordem $m$;
- $G_i(\mathbf{p_k}) = exp(-0.5 (\mathbf{p_k}-\mathbf{\mu}^{2D}_i)^T \Sigma_i^{'-1} (\mathbf{p_k}-\mathbf{\mu}^{2D}_i))$;

- $\mu^{2D}_{i} = \begin{bmatrix}\frac{μ^{3D}_{ix} {P}_{0,0} + μ^{3D}_{iy} {P}_{0,1} + μ^{3D}_{iz} {P}_{0,2} + {P}_{0,3}}{μ^{3D}_{ix} {P}_{2,0} + μ^{3D}_{iy} {P}_{2,1} + μ^{3D}_{iz} {P}_{2,2} + {P}_{2,3}}\\\frac{μ^{3D}_{ix} {P}_{1,0} + μ^{3D}_{iy} {P}_{1,1} + μ^{3D}_{iz} {P}_{1,2} + {P}_{1,3}}{μ^{3D}_{ix} {P}_{2,0} + μ^{3D}_{iy} {P}_{2,1} + μ^{3D}_{iz} {P}_{2,2} + {P}_{2,3}}\end{bmatrix}$ , $P_{m,n}$ são as entradas de uma matriz $P \in \mathbb{R}^{3 \times 4}$;
- $\Sigma_i' = JWR_iS_iS_i^TR_i^TW^TJ^T$;
- $S_i = diag(\mathbf{s}_{i}) = \begin{bmatrix} s_{ix} & 0 & 0 \\ 0 & s_{iy} &0 \\ 0 & 0 & s_{iz} \end{bmatrix}$;
- $J \in \mathbb{R}^{2 \times 3}$ e $W,R_i \in \mathbb{R}^{3 \times 3}$;
- ${k_{li}^m} \in \mathbb{R}$, $\mathbf{\mu}^{2D}_i,\mathbf{p_k} \in \mathbb{R}^2$ e $cp, \mathbf{\mu}^{3D}_i, \mathbf{s}_{i} \in \mathbb{R}^3$;
- $\mathbf{\mu}^{3D}_{i}, {k_{li}^m}$ e $\mathbf{s}_{i}$ serão atualizados pelo gradiente descendente.

$\displaystyle \nabla(\mu^{3D}) = \frac{\partial \mathcal{L}}{\partial \mu^{3D}} =\frac{\partial}{\partial \mu^{3D}}\left(\frac{2\overline{C}\ \overline{C'}+ 10^{-9}}{\overline{C}^2 + \overline{C'}^2 + 10^{-9}}\right) \cdot \left(\frac{2\sigma_{CC'}+ 10^{-9}}{\sigma_{C}^2 + \sigma_{C'}^2 + 10^{-9}}\right) + \left(\frac{2\overline{C}\ \overline{C'}+ 10^{-9}}{\overline{C}^2 + \overline{C'}^2 + 10^{-9}}\right) \frac{\partial}{\partial \mu^{3D}} \left(\frac{2\sigma_{CC'}+ 10^{-9}}{\sigma_{C}^2 + \sigma_{C'}^2 + 10^{-9}}\right)$

$\displaystyle \nabla(\mathbf{s}) = \frac{\partial \mathcal{L}}{\partial \mathbf{s}}  = \frac{\partial}{\partial \mathbf{s}}\left(\frac{2\overline{C}\ \overline{C'}+ 10^{-9}}{\overline{C}^2 + \overline{C'}^2 + 10^{-9}}\right) \cdot \left(\frac{2\sigma_{CC'}+ 10^{-9}}{\sigma_{C}^2 + \sigma_{C'}^2 + 10^{-9}}\right) + \left(\frac{2\overline{C}\ \overline{C'}+ 10^{-9}}{\overline{C}^2 + \overline{C'}^2 + 10^{-9}}\right) \frac{\partial}{\partial \mathbf{s}} \left(\frac{2\sigma_{CC'}+ 10^{-9}}{\sigma_{C}^2 + \sigma_{C'}^2 + 10^{-9}}\right)$

$\displaystyle \nabla(k) = \frac{\partial \mathcal{L}}{\partial k}  = \frac{\partial}{\partial k}\left(\frac{2\overline{C}\ \overline{C'}+ 10^{-9}}{\overline{C}^2 + \overline{C'}^2 + 10^{-9}}\right) \cdot \left(\frac{2\sigma_{CC'}+ 10^{-9}}{\sigma_{C}^2 + \sigma_{C'}^2 + 10^{-9}}\right) + \left(\frac{2\overline{C}\ \overline{C'}+ 10^{-9}}{\overline{C}^2 + \overline{C'}^2 + 10^{-9}}\right) \frac{\partial}{\partial k} \left(\frac{2\sigma_{CC'}+ 10^{-9}}{\sigma_{C}^2 + \sigma_{C'}^2 + 10^{-9}}\right)$

Para $N=3, M=4, C'(\mathbf{p_k}) = k, l_{max}=1, k_{li}^m=1/(i+1), cp = [0,0,0], \ \mathbf{\mu^{3D}_i}=[1+i,1+i,1+i],\ \mathbf{s_i} = [0.5(i+1),0.5(i+1),0.5(i+1)]$, $J = diag(1,1,0), P = diag(1,1,1,0), W = I, R_i=I \rightarrow$

$\nabla(\mu) = ?$

$\nabla(\mathbf{s}) = ?$

$\nabla(k) = ?$


#### Implementação SymPy:

In [75]:
import sympy as sp
import sympy.stats as stats
from tqdm.notebook import tqdm

def mean(expr_list):
    mean_expr = sum(expr_list) / len(expr_list)
    return mean_expr 

def variance(expr_list1, mean_expr1, expr_list2=None, mean_expr2=None):
    if not expr_list2:
        variance_expr = sum((x - mean_expr1)**2 for x in expr_list1) / (len(expr_list1)-1)
    else:
        variance_expr = sum((x - mean_expr1)*(y - mean_expr2) for x,y in zip(expr_list1, expr_list2) ) / (len(expr_list1)-1)
    return variance_expr 

def dssim_loss(img1, img2):
    mu_img1 = mean(img1)
    mu_img2 = mean(img2)
    var_img1 = variance(img1, mu_img1)
    var_img2 = variance(img2, mu_img2)
    cov_img1img2 = variance(img1, mu_img1, img2, mu_img2)
    print(cov_img1img2.subs(values).evalf())
    ssim = (2*mu_img1*mu_img2 +10e-9)/(mu_img1**2 + mu_img2**2 + 10e-9)
    ssim *= (2*cov_img1img2 + 10e-9)/(var_img1**2 + var_img2**2 + 10e-9)
    return 1 - ssim

def build_color(sh, rays_d):
    lmax = 1  # Maximum spherical harmonics degree
    C0 = 0.28209479177387814
    C1 = 0.4886025119029199

    color = C0 * sh[:, 0][0]
    if lmax > 0:
        x, y, z = rays_d[0, :][0], rays_d[1, :][0], rays_d[2, :][0]
        color = (color -
                C1 * y * sh[:, 1][0] +
                C1 * z * sh[:, 2][0] -
                C1 * x * sh[:, 3][0])
    
    color = (color + 0.5)
    return color

N = 3  # Number of gaussians
M = 4 # Number of pixels
R_i = sp.Matrix([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0],[0.0, 0.0, 1.0]]) # assumir conhecida por simplicidade
W = sp.Matrix([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0],[0.0, 0.0, 1.0]])
J = sp.Matrix([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0]])
P = sp.Matrix([[1000.0, 0.0, 500.0, 300.0], [0.0, 1000.0, 500.0, 300.0], [300.0, 300.0, 300.0, 300.0]])
cp = sp.Matrix([0.0, 0.0, 0.0])

# variam para n=1,...,N
sh0 = sp.IndexedBase('sh_0') # k_0^0   <-- grau_max = 0 
sh1 = sp.IndexedBase('sh_1') # k_1^{-1}
sh2 = sp.IndexedBase('sh_2') # k_1^{0}
sh3 = sp.IndexedBase('sh_3') # k_1^{1} <-- grau_max = 1 

mu3Dx = sp.IndexedBase('μ_x')
mu3Dy = sp.IndexedBase('μ_y')
mu3Dz = sp.IndexedBase('μ_z')

sx = sp.IndexedBase('s_x')
sy = sp.IndexedBase('s_y')
sz = sp.IndexedBase('s_z')

# Coordenada do ponto de pesquisa
p_k = sp.Matrix([[i, j] for i in range(int(M**(0.5))) for j in range(int(M**(0.5)))])

# Observações
C_hat = sp.Matrix([[k] for k in range(M)])

# Generate values to apply to the derivative
values = {}
for i in range(N):
    values = values | {mu3Dx[i]: 1+i, mu3Dy[i]: 1+i, mu3Dz[i]: 1+i,
                       sx[i]:0.5*(i+1), sy[i]:0.5*(i+1), sz[i]:0.5*(i+1),
                       sh0[i]:1/(i+1),sh1[i]:1/(i+1), sh2[i]:1/(i+1), sh3[i]:1/(i+1) }

#Forward pass
def C_(p):
    # Expression for C(x)
    C_values = 0 
    for i in range(0,N):   
        sh_i = sp.Matrix([sh0[i], sh1[i], sh2[i], sh3[i]])
        mu3D_i = sp.Matrix([mu3Dx[i], mu3Dy[i], mu3Dz[i]])
        mu2Dx_i = (mu3Dx[i] * P[0, 0] + mu3Dy[i] * P[0, 1] + mu3Dz[i] * P[0, 2] + P[0, 3])/(mu3Dx[i] * P[2, 0] + mu3Dy[i] * P[2,1] + mu3Dz[i] * P[2, 2] + P[2, 3])
        mu2Dy_i = (mu3Dx[i] * P[1, 0] + mu3Dy[i] * P[1, 1] + mu3Dz[i] * P[1, 2] + P[1, 3])/(mu3Dx[i] * P[2, 0] + mu3Dy[i] * P[2,1] + mu3Dz[i] * P[2, 2] + P[2, 3])
        d_i = p - sp.Matrix([mu2Dx_i, mu2Dy_i])
        S_i = sp.Matrix([[sx[i], 0, 0], [0, sy[i], 0], [0, 0, sz[i]]])
        Sigma_2D_i = J * W * R_i * S_i * S_i.T * R_i.T * W.T * J.T
        Sigma_2D_inv_i = Sigma_2D_i.inv()
        G_i = sp.exp(-0.5 * (d_i.T * Sigma_2D_inv_i * d_i)[0, 0])
        prod = 1
        for j in range(0, i):
            mu2Dx_j = (mu3Dx[j] * P[0, 0] + mu3Dy[j] * P[0, 1] + mu3Dz[j] * P[0, 2] + P[0, 3])/(mu3Dx[j] * P[2, 0] + mu3Dy[j] * P[2,1] + mu3Dz[j] * P[2, 2] + P[2, 3])
            mu2Dy_j = (mu3Dx[j] * P[1, 0] + mu3Dy[j] * P[1, 1] + mu3Dz[j] * P[1, 2] + P[1, 3])/(mu3Dx[j] * P[2, 0] + mu3Dy[j] * P[2,1] + mu3Dz[j] * P[2, 2] + P[2, 3])
            d_j = p - sp.Matrix([mu2Dx_j, mu2Dy_j])
            S_j = sp.Matrix([[sx[j], 0, 0], [0, sy[j], 0], [0, 0, sz[j]]])
            Sigma_2D_j = J * W * R_i * S_j * S_j.T * R_i.T * W.T * J.T
            Sigma_2D_inv_j = Sigma_2D_j.inv()
            G_j = sp.exp(-0.5 * (d_j.T * Sigma_2D_inv_j * d_j)[0, 0])
            prod *= (1 - G_j)
        direction = mu3D_i - cp
        c_i = build_color(sh_i.T, direction)
        C_values += c_i * G_i * prod
    return C_values

C = []
for k in tqdm(range(M)):
    C.append( C_(p_k.row(k).T) )
C = sp.Matrix(C)

L = dssim_loss(C, C_hat)

print("L=")
# display(L) # slow

print(f"Evaluation data: {values}:")
print(f"L evaluated:", L.subs(values).evalf())

## Sympy can't handle these sizes of derivatives...
dL_dmu = []
for i in tqdm(range(N)):
    dL_dmu_i = sp.diff(L, sp.Matrix([mu3Dx[i], mu3Dy[i], mu3Dz[i]]))
    dL_dmu_num = dL_dmu_i.subs(values).evalf().tolist()
    print(f"Derivative w.r.t μ_{i} evaluated:", dL_dmu_num )
    dL_dmu.append(dL_dmu_num)

dL_ds = []
for i in tqdm(range(N)):
    dL_ds_i = sp.diff(L, sp.Matrix([sx[i], sy[i], sz[i]]))
    dL_ds_num = dL_ds_i.subs(values).evalf().tolist()
    print(f"Derivative w.r.t s_{i} evaluated:", dL_ds_num )
    dL_ds.append(dL_ds_num)

dL_dsh = []
for i in tqdm(range(N)):
    dL_dsh_i = sp.diff(L, sp.Matrix([sh0[i], sh1[i], sh2[i], sh3[i]]))
    dL_dsh_num = dL_dsh_i.subs(values).evalf().tolist()
    print(f"Derivative w.r.t sh_{i} evaluated:",dL_dsh_num )
    dL_dsh.append(dL_dsh_num)

  0%|          | 0/4 [00:00<?, ?it/s]

0.0747213673008819
L=
Evaluation data: {μ_x[0]: 1, μ_y[0]: 1, μ_z[0]: 1, s_x[0]: 0.5, s_y[0]: 0.5, s_z[0]: 0.5, sh_0[0]: 1.0, sh_1[0]: 1.0, sh_2[0]: 1.0, sh_3[0]: 1.0, μ_x[1]: 2, μ_y[1]: 2, μ_z[1]: 2, s_x[1]: 1.0, s_y[1]: 1.0, s_z[1]: 1.0, sh_0[1]: 0.5, sh_1[1]: 0.5, sh_2[1]: 0.5, sh_3[1]: 0.5, μ_x[2]: 3, μ_y[2]: 3, μ_z[2]: 3, s_x[2]: 1.5, s_y[2]: 1.5, s_z[2]: 1.5, sh_0[2]: 0.3333333333333333, sh_1[2]: 0.3333333333333333, sh_2[2]: 0.3333333333333333, sh_3[2]: 0.3333333333333333}:
L evaluated: 0.992887347678695


  0%|          | 0/3 [00:00<?, ?it/s]

Derivative w.r.t μ_0 evaluated: [[0.0124251889940151], [0.0125779356047527], [-0.0111994942393178]]
Derivative w.r.t μ_1 evaluated: [[0.00863845206681180], [0.00899944377054137], [-0.00873734297318112]]
Derivative w.r.t μ_2 evaluated: [[0.00232789691964532], [0.00252158995460855], [-0.00241591817895880]]


  0%|          | 0/3 [00:00<?, ?it/s]

Derivative w.r.t s_0 evaluated: [[-0.00774147418890434], [-0.00847465792044467], [0]]
Derivative w.r.t s_1 evaluated: [[-0.000237352604004352], [-0.00186181527078744], [0]]
Derivative w.r.t s_2 evaluated: [[0.000386389861524028], [-0.000465859492314193], [0]]


  0%|          | 0/3 [00:00<?, ?it/s]

Derivative w.r.t sh_0 evaluated: [[-0.00684190568634900], [0.0118505282693509], [-0.0118505282693509], [0.0118505282693509]]
Derivative w.r.t sh_1 evaluated: [[-0.0101361292723854], [0.0351125817837154], [-0.0351125817837154], [0.0351125817837154]]
Derivative w.r.t sh_2 evaluated: [[-0.00419213593065449], [0.0217829772723858], [-0.0217829772723858], [0.0217829772723858]]


#### Implementação PyTorch:

In [91]:
import torch

def dssim_loss(img1, img2):
    mu_img1 = torch.mean(img1)
    mu_img2 = torch.mean(img2)
    var_img1 = torch.var(img1)
    var_img2 = torch.var(img2)
    cov_img1img2 = ((img1 - mu_img1) * (img2 - mu_img2)).sum() / (img1.size(0) - 1)
    ssim = (2*mu_img1*mu_img2 +10e-9)/(mu_img1**2 + mu_img2**2 + 10e-9)
    ssim *= (2*cov_img1img2 + 10e-9)/(var_img1**2 + var_img2**2 + 10e-9)
    return 1 - ssim

def build_color(sh_i, rays_d):
    lmax = 1  # Maximum spherical harmonics degree
    C0 = 0.28209479177387814
    C1 = 0.4886025119029199
    color = C0 * sh_i[..., 0]
    if lmax > 0:
        x, y, z = rays_d[0, ...], rays_d[1, ...], rays_d[2, ...]
        color = (color -
                C1 * y * sh_i[..., 1] +
                C1 * z * sh_i[..., 2] -
                C1 * x * sh_i[..., 3])
    
    color = (color + 0.5)
    return color

N = 3  # Number of gaussians
M = 4 # Number of pixels

R_i = torch.tensor([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0],[0.0, 0.0, 1.0]], requires_grad=False)
W = torch.tensor([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0],[0.0, 0.0, 1.0]], requires_grad=False)
J = torch.tensor([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0]], requires_grad=False)
P = torch.tensor([[1000.0, 0.0, 500.0, 300.0], [0.0, 1000.0, 500.0, 300.0], [300.0, 300.0, 300.0, 300.0]], requires_grad=False)
cp = torch.tensor([0.0, 0.0, 0.0], requires_grad=False)

# variam para n=1,...,N
mu3D = torch.tensor([[1.0+i, 1.0+i, 1.0+i] for i in range(N)], requires_grad=True)
s = torch.tensor([[0.5*(i+1), 0.5*(i+1), 0.5*(i+1)] for i in range(N)], requires_grad=True) 
sh = torch.tensor([[1.0/(i+1), 1.0/(i+1), 1.0/(i+1), 1.0/(i+1)] for i in range(N)], requires_grad=True)

# Coordenada do ponto de pesquisa
p_k = torch.tensor([[i, j] for i in range(int(M**(0.5))) for j in range(int(M**(0.5)))], requires_grad=False)

# Observações
C_hat = torch.tensor([k for k in range(M)], dtype=torch.float32)

#Forward pass
def C_(p):
    # Expression for C(x)
    C_values = torch.tensor(0.0) 
    for i in range(N):
        mu2Dx_i = (mu3D[i,0] * P[0, 0] + mu3D[i,1] * P[0, 1] + mu3D[i,2] * P[0, 2] + P[0, 3])/(mu3D[i,0] * P[2, 0] + mu3D[i,1] * P[2,1] + mu3D[i,2] * P[2, 2] + P[2, 3])
        mu2Dy_i = (mu3D[i,0] * P[1, 0] + mu3D[i,1] * P[1, 1] + mu3D[i,2] * P[1, 2] + P[1, 3])/(mu3D[i,0] * P[2, 0] + mu3D[i,1] * P[2,1] + mu3D[i,2] * P[2, 2] + P[2, 3])
        d_i = p - torch.stack([mu2Dx_i, mu2Dy_i])
        S_i = torch.diag(s[i])
        Sigma_2D_i = J @ W @ R_i @ S_i @ S_i.T @ R_i.T @ W.T @ J.T
        Sigma_2D_inv_i = torch.linalg.inv(Sigma_2D_i)
        G_i = torch.exp(-0.5 * (d_i.T @ Sigma_2D_inv_i @ d_i))
        prod = 1
        for j in range(i):
            mu2Dx_j = (mu3D[j,0] * P[0, 0] + mu3D[j,1] * P[0, 1] + mu3D[j,2] * P[0, 2] + P[0, 3])/(mu3D[j,0] * P[2, 0] + mu3D[j,1] * P[2,1] + mu3D[j,2] * P[2, 2] + P[2, 3])
            mu2Dy_j = (mu3D[j,0] * P[1, 0] + mu3D[j,1] * P[1, 1] + mu3D[j,2] * P[1, 2] + P[1, 3])/(mu3D[j,0] * P[2, 0] + mu3D[j,1] * P[2,1] + mu3D[j,2] * P[2, 2] + P[2, 3])
            d_j = p - torch.stack([mu2Dx_j, mu2Dy_j])
            S_j = torch.diag(s[j])
            Sigma_2D_j = J @ W @ R_i @ S_j @ S_j.T @ R_i.T @ W.T @ J.T
            Sigma_2D_inv_j = torch.linalg.inv(Sigma_2D_j)
            G_j = torch.exp(-0.5 * (d_j.T @ Sigma_2D_inv_j @ d_j))
            prod *= (1 - G_j)
        direction = mu3D[i] - cp
        c_i = build_color(sh[i], direction)
        C_values += c_i * G_i * prod
    return C_values

C = []
for k in tqdm(range(M)):
    C.append( C_(p_k[k]) )
C = torch.stack(C)

L = dssim_loss( C , C_hat)
print("L=", L)

# Backward pass
L.backward()

# Gradients
print(mu3D.grad)
print(s.grad)
print(sh.grad)


  0%|          | 0/4 [00:00<?, ?it/s]

L= tensor(0.9929, grad_fn=<RsubBackward1>)
tensor([[ 0.0124,  0.0126, -0.0112],
        [ 0.0086,  0.0090, -0.0087],
        [ 0.0023,  0.0025, -0.0024]])
tensor([[-0.0077, -0.0085,  0.0000],
        [-0.0002, -0.0019,  0.0000],
        [ 0.0004, -0.0005,  0.0000]])
tensor([[-0.0068,  0.0119, -0.0119,  0.0119],
        [-0.0101,  0.0351, -0.0351,  0.0351],
        [-0.0042,  0.0218, -0.0218,  0.0218]])


#### Comparação:

In [76]:
dL_dmu = torch.tensor(dL_dmu, dtype=torch.float32).reshape(mu3D.grad.shape)
dL_ds = torch.tensor(dL_ds, dtype=torch.float32).reshape(s.grad.shape)
dL_dsh = torch.tensor(dL_dsh, dtype=torch.float32).reshape(sh.grad.shape)

# Gradients
print(mu3D.grad, "=\n", dL_dmu, "\n")
print(s.grad, "=\n", dL_ds, "\n" )
print(sh.grad, "=\n", dL_dsh, "\n" )

tensor([[ 0.0124,  0.0126, -0.0112],
        [ 0.0086,  0.0090, -0.0087],
        [ 0.0023,  0.0025, -0.0024]]) =
 tensor([[ 0.0124,  0.0126, -0.0112],
        [ 0.0086,  0.0090, -0.0087],
        [ 0.0023,  0.0025, -0.0024]]) 

tensor([[-0.0077, -0.0085,  0.0000],
        [-0.0002, -0.0019,  0.0000],
        [ 0.0004, -0.0005,  0.0000]]) =
 tensor([[-0.0077, -0.0085,  0.0000],
        [-0.0002, -0.0019,  0.0000],
        [ 0.0004, -0.0005,  0.0000]]) 

tensor([[-0.0068,  0.0119, -0.0119,  0.0119],
        [-0.0101,  0.0351, -0.0351,  0.0351],
        [-0.0042,  0.0218, -0.0218,  0.0218]]) =
 tensor([[-0.0068,  0.0119, -0.0119,  0.0119],
        [-0.0101,  0.0351, -0.0351,  0.0351],
        [-0.0042,  0.0218, -0.0218,  0.0218]]) 



# Exemplo 16

$\displaystyle \mathcal{L}(C, C') = L_{recon}(C, C') + L_{DSSIM}(C, C')$

- $C$ e $C'$ são as imagens, reconstruída e prevista respectivamente, vetorizadas com $M$ pixels cada
- $\displaystyle L_{recon}(C, C') =\sum_{k=1}^{M} || C(\mathbf{p_k}) - C'(\mathbf{p_k}) ||^2 $

- $\displaystyle L_{DSSIM}(C, C') = 1- \left(\frac{2\overline{C}\overline{C'}+ 10^{-9}}{\overline{C}^2 + \overline{C'}^2 + 10^{-9}}\right) \cdot \left(\frac{2\sigma_{CC'}+ 10^{-9}}{\sigma_{C}^2 + \sigma_{C'}^2 + 10^{-9}}\right)$
- $\mathbf{p_k}$ é a coordenada do pixel de posição $k$ em cada imagem vetorizada
- $\overline{C}$, $\overline{C'}$ são as médias de $C$ e $C'$
- $\sigma_{C}$, $\sigma_{C'}$ são as variâncias de $C$ e $C'$
- $\sigma_{CC'}$ é a covariância de $C$ e $C'$


#### Implementação SymPy

In [86]:
import sympy as sp
import sympy.stats as stats
from tqdm.notebook import tqdm

def mean(expr_list):
    mean_expr = sum(expr_list) / len(expr_list)
    return mean_expr 

def variance(expr_list1, mean_expr1, expr_list2=None, mean_expr2=None):
    if not expr_list2:
        variance_expr = sum((x - mean_expr1)**2 for x in expr_list1) / (len(expr_list1)-1)
    else:
        variance_expr = sum((x - mean_expr1)*(y - mean_expr2) for x,y in zip(expr_list1, expr_list2) ) / (len(expr_list1)-1)
    return variance_expr 

def dssim_loss(img1, img2):
    mu_img1 = mean(img1)
    mu_img2 = mean(img2)
    var_img1 = variance(img1, mu_img1)
    var_img2 = variance(img2, mu_img2)
    cov_img1img2 = variance(img1, mu_img1, img2, mu_img2)
    ssim = (2*mu_img1*mu_img2 +10e-9)/(mu_img1**2 + mu_img2**2 + 10e-9)
    ssim *= (2*cov_img1img2 + 10e-9)/(var_img1**2 + var_img2**2 + 10e-9)
    return 1 - ssim

def build_color(sh, rays_d):
    lmax = 1  # Maximum spherical harmonics degree
    C0 = 0.28209479177387814
    C1 = 0.4886025119029199

    color = C0 * sh[:, 0][0]
    if lmax > 0:
        x, y, z = rays_d[0, :][0], rays_d[1, :][0], rays_d[2, :][0]
        color = (color -
                C1 * y * sh[:, 1][0] +
                C1 * z * sh[:, 2][0] -
                C1 * x * sh[:, 3][0])
    
    color = (color + 0.5)
    return color

N = 3  # Number of gaussians
M = 4 # Number of pixels
R_i = sp.Matrix([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0],[0.0, 0.0, 1.0]]) # assumir conhecida por simplicidade
W = sp.Matrix([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0],[0.0, 0.0, 1.0]])
J = sp.Matrix([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0]])
P = sp.Matrix([[1000.0, 0.0, 500.0, 300.0], [0.0, 1000.0, 500.0, 300.0], [300.0, 300.0, 300.0, 300.0]])
cp = sp.Matrix([0.0, 0.0, 0.0])

# variam para n=1,...,N
sh0 = sp.IndexedBase('sh_0') # k_0^0   <-- grau_max = 0 
sh1 = sp.IndexedBase('sh_1') # k_1^{-1}
sh2 = sp.IndexedBase('sh_2') # k_1^{0}
sh3 = sp.IndexedBase('sh_3') # k_1^{1} <-- grau_max = 1 

mu3Dx = sp.IndexedBase('μ_x')
mu3Dy = sp.IndexedBase('μ_y')
mu3Dz = sp.IndexedBase('μ_z')

sx = sp.IndexedBase('s_x')
sy = sp.IndexedBase('s_y')
sz = sp.IndexedBase('s_z')

# Coordenada do ponto de pesquisa
p_k = sp.Matrix([[i, j] for i in range(int(M**(0.5))) for j in range(int(M**(0.5)))])

# Observações
C_hat = sp.Matrix([[k] for k in range(M)])

# Generate values to apply to the derivative
values = {}
for i in range(N):
    values = values | {mu3Dx[i]: 1+i, mu3Dy[i]: 1+i, mu3Dz[i]: 1+i,
                       sx[i]:0.5*(i+1), sy[i]:0.5*(i+1), sz[i]:0.5*(i+1),
                       sh0[i]:1/(i+1),sh1[i]:1/(i+1), sh2[i]:1/(i+1), sh3[i]:1/(i+1) }

#Forward pass
def C_(p):
    # Expression for C(x)
    C_values = 0 
    for i in range(0,N):   
        sh_i = sp.Matrix([sh0[i], sh1[i], sh2[i], sh3[i]])
        mu3D_i = sp.Matrix([mu3Dx[i], mu3Dy[i], mu3Dz[i]])
        mu2Dx_i = (mu3Dx[i] * P[0, 0] + mu3Dy[i] * P[0, 1] + mu3Dz[i] * P[0, 2] + P[0, 3])/(mu3Dx[i] * P[2, 0] + mu3Dy[i] * P[2,1] + mu3Dz[i] * P[2, 2] + P[2, 3])
        mu2Dy_i = (mu3Dx[i] * P[1, 0] + mu3Dy[i] * P[1, 1] + mu3Dz[i] * P[1, 2] + P[1, 3])/(mu3Dx[i] * P[2, 0] + mu3Dy[i] * P[2,1] + mu3Dz[i] * P[2, 2] + P[2, 3])
        d_i = p - sp.Matrix([mu2Dx_i, mu2Dy_i])
        S_i = sp.Matrix([[sx[i], 0, 0], [0, sy[i], 0], [0, 0, sz[i]]])
        Sigma_2D_i = J * W * R_i * S_i * S_i.T * R_i.T * W.T * J.T
        Sigma_2D_inv_i = Sigma_2D_i.inv()
        G_i = sp.exp(-0.5 * (d_i.T * Sigma_2D_inv_i * d_i)[0, 0])
        prod = 1
        for j in range(0, i):
            mu2Dx_j = (mu3Dx[j] * P[0, 0] + mu3Dy[j] * P[0, 1] + mu3Dz[j] * P[0, 2] + P[0, 3])/(mu3Dx[j] * P[2, 0] + mu3Dy[j] * P[2,1] + mu3Dz[j] * P[2, 2] + P[2, 3])
            mu2Dy_j = (mu3Dx[j] * P[1, 0] + mu3Dy[j] * P[1, 1] + mu3Dz[j] * P[1, 2] + P[1, 3])/(mu3Dx[j] * P[2, 0] + mu3Dy[j] * P[2,1] + mu3Dz[j] * P[2, 2] + P[2, 3])
            d_j = p - sp.Matrix([mu2Dx_j, mu2Dy_j])
            S_j = sp.Matrix([[sx[j], 0, 0], [0, sy[j], 0], [0, 0, sz[j]]])
            Sigma_2D_j = J * W * R_i * S_j * S_j.T * R_i.T * W.T * J.T
            Sigma_2D_inv_j = Sigma_2D_j.inv()
            G_j = sp.exp(-0.5 * (d_j.T * Sigma_2D_inv_j * d_j)[0, 0])
            prod *= (1 - G_j)
        direction = mu3D_i - cp
        c_i = build_color(sh_i.T, direction)
        C_values += c_i * G_i * prod
    return C_values

L1 = 0   
C = []
for k in tqdm(range(M)):
    C.append( C_(p_k.row(k).T) )
    L1 += (C[-1] - C_hat[k])**2
C = sp.Matrix(C)

L2 = dssim_loss(C, C_hat)

L = L2 #+L2
print("L=")
# display(L) # slow

print(f"Evaluation data: {values}:")
print(f"L evaluated:", L.subs(values).evalf())

# Sympy can't handle these sizes of derivatives...
dL_dmu = []
for i in tqdm(range(N)):
    dL_dmu_i = sp.diff(L, sp.Matrix([mu3Dx[i], mu3Dy[i], mu3Dz[i]]))
    dL_dmu_num = dL_dmu_i.subs(values).evalf().tolist()
    print(f"Derivative w.r.t μ_{i} evaluated:", dL_dmu_num )
    dL_dmu.append(dL_dmu_num)

dL_ds = []
for i in tqdm(range(N)):
    dL_ds_i = sp.diff(L, sp.Matrix([sx[i], sy[i], sz[i]]))
    dL_ds_num = dL_ds_i.subs(values).evalf().tolist()
    print(f"Derivative w.r.t s_{i} evaluated:", dL_ds_num )
    dL_ds.append(dL_ds_num)

dL_dsh = []
for i in tqdm(range(N)):
    dL_dsh_i = sp.diff(L, sp.Matrix([sh0[i], sh1[i], sh2[i], sh3[i]]))
    dL_dsh_num = dL_dsh_i.subs(values).evalf().tolist()
    print(f"Derivative w.r.t sh_{i} evaluated:",dL_dsh_num )
    dL_dsh.append(dL_dsh_num)

  0%|          | 0/4 [00:00<?, ?it/s]

L=
Evaluation data: {μ_x[0]: 1, μ_y[0]: 1, μ_z[0]: 1, s_x[0]: 0.5, s_y[0]: 0.5, s_z[0]: 0.5, sh_0[0]: 1.0, sh_1[0]: 1.0, sh_2[0]: 1.0, sh_3[0]: 1.0, μ_x[1]: 2, μ_y[1]: 2, μ_z[1]: 2, s_x[1]: 1.0, s_y[1]: 1.0, s_z[1]: 1.0, sh_0[1]: 0.5, sh_1[1]: 0.5, sh_2[1]: 0.5, sh_3[1]: 0.5, μ_x[2]: 3, μ_y[2]: 3, μ_z[2]: 3, s_x[2]: 1.5, s_y[2]: 1.5, s_z[2]: 1.5, sh_0[2]: 0.3333333333333333, sh_1[2]: 0.3333333333333333, sh_2[2]: 0.3333333333333333, sh_3[2]: 0.3333333333333333}:
L evaluated: 0.992887347678695


  0%|          | 0/3 [00:00<?, ?it/s]

Derivative w.r.t μ_0 evaluated: [[0.0124251889940151], [0.0125779356047527], [-0.0111994942393178]]
Derivative w.r.t μ_1 evaluated: [[0.00863845206681180], [0.00899944377054137], [-0.00873734297318112]]
Derivative w.r.t μ_2 evaluated: [[0.00232789691964532], [0.00252158995460855], [-0.00241591817895880]]


  0%|          | 0/3 [00:00<?, ?it/s]

Derivative w.r.t s_0 evaluated: [[-0.00774147418890434], [-0.00847465792044467], [0]]
Derivative w.r.t s_1 evaluated: [[-0.000237352604004352], [-0.00186181527078744], [0]]
Derivative w.r.t s_2 evaluated: [[0.000386389861524028], [-0.000465859492314193], [0]]


  0%|          | 0/3 [00:00<?, ?it/s]

Derivative w.r.t sh_0 evaluated: [[-0.00684190568634900], [0.0118505282693509], [-0.0118505282693509], [0.0118505282693509]]
Derivative w.r.t sh_1 evaluated: [[-0.0101361292723854], [0.0351125817837154], [-0.0351125817837154], [0.0351125817837154]]
Derivative w.r.t sh_2 evaluated: [[-0.00419213593065449], [0.0217829772723858], [-0.0217829772723858], [0.0217829772723858]]


#### Implementação PyTorch:

In [92]:
import torch

def dssim_loss(img1, img2):
    mu_img1 = torch.mean(img1)
    mu_img2 = torch.mean(img2)
    var_img1 = torch.var(img1)
    var_img2 = torch.var(img2)
    cov_img1img2 = ((img1 - mu_img1) * (img2 - mu_img2)).sum() / (img1.size(0) - 1)
    ssim = (2*mu_img1*mu_img2 +10e-9)/(mu_img1**2 + mu_img2**2 + 10e-9)
    ssim *= (2*cov_img1img2 + 10e-9)/(var_img1**2 + var_img2**2 + 10e-9)
    return 1 - ssim

def build_color(sh_i, rays_d):
    lmax = 1  # Maximum spherical harmonics degree
    C0 = 0.28209479177387814
    C1 = 0.4886025119029199
    color = C0 * sh_i[..., 0]
    if lmax > 0:
        x, y, z = rays_d[0, ...], rays_d[1, ...], rays_d[2, ...]
        color = (color -
                C1 * y * sh_i[..., 1] +
                C1 * z * sh_i[..., 2] -
                C1 * x * sh_i[..., 3])
    
    color = (color + 0.5)
    return color

N = 3  # Number of gaussians
M = 4 # Number of pixels

R_i = torch.tensor([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0],[0.0, 0.0, 1.0]], requires_grad=False)
W = torch.tensor([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0],[0.0, 0.0, 1.0]], requires_grad=False)
J = torch.tensor([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0]], requires_grad=False)
P = torch.tensor([[1000.0, 0.0, 500.0, 300.0], [0.0, 1000.0, 500.0, 300.0], [300.0, 300.0, 300.0, 300.0]], requires_grad=False)
cp = torch.tensor([0.0, 0.0, 0.0], requires_grad=False)

# variam para n=1,...,N
mu3D = torch.tensor([[1.0+i, 1.0+i, 1.0+i] for i in range(N)], requires_grad=True)
s = torch.tensor([[0.5*(i+1), 0.5*(i+1), 0.5*(i+1)] for i in range(N)], requires_grad=True) 
sh = torch.tensor([[1.0/(i+1), 1.0/(i+1), 1.0/(i+1), 1.0/(i+1)] for i in range(N)], requires_grad=True)

# Coordenada do ponto de pesquisa
p_k = torch.tensor([[i, j] for i in range(int(M**(0.5))) for j in range(int(M**(0.5)))], requires_grad=False)

# Observações
C_hat = torch.tensor([k for k in range(M)], dtype=torch.float32)

#Forward pass
def C_(p):
    # Expression for C(x)
    C_values = torch.tensor(0.0) 
    for i in range(N):
        mu2Dx_i = (mu3D[i,0] * P[0, 0] + mu3D[i,1] * P[0, 1] + mu3D[i,2] * P[0, 2] + P[0, 3])/(mu3D[i,0] * P[2, 0] + mu3D[i,1] * P[2,1] + mu3D[i,2] * P[2, 2] + P[2, 3])
        mu2Dy_i = (mu3D[i,0] * P[1, 0] + mu3D[i,1] * P[1, 1] + mu3D[i,2] * P[1, 2] + P[1, 3])/(mu3D[i,0] * P[2, 0] + mu3D[i,1] * P[2,1] + mu3D[i,2] * P[2, 2] + P[2, 3])
        d_i = p - torch.stack([mu2Dx_i, mu2Dy_i])
        S_i = torch.diag(s[i])
        Sigma_2D_i = J @ W @ R_i @ S_i @ S_i.T @ R_i.T @ W.T @ J.T
        Sigma_2D_inv_i = torch.linalg.inv(Sigma_2D_i)
        G_i = torch.exp(-0.5 * (d_i.T @ Sigma_2D_inv_i @ d_i))
        prod = 1
        for j in range(i):
            mu2Dx_j = (mu3D[j,0] * P[0, 0] + mu3D[j,1] * P[0, 1] + mu3D[j,2] * P[0, 2] + P[0, 3])/(mu3D[j,0] * P[2, 0] + mu3D[j,1] * P[2,1] + mu3D[j,2] * P[2, 2] + P[2, 3])
            mu2Dy_j = (mu3D[j,0] * P[1, 0] + mu3D[j,1] * P[1, 1] + mu3D[j,2] * P[1, 2] + P[1, 3])/(mu3D[j,0] * P[2, 0] + mu3D[j,1] * P[2,1] + mu3D[j,2] * P[2, 2] + P[2, 3])
            d_j = p - torch.stack([mu2Dx_j, mu2Dy_j])
            S_j = torch.diag(s[j])
            Sigma_2D_j = J @ W @ R_i @ S_j @ S_j.T @ R_i.T @ W.T @ J.T
            Sigma_2D_inv_j = torch.linalg.inv(Sigma_2D_j)
            G_j = torch.exp(-0.5 * (d_j.T @ Sigma_2D_inv_j @ d_j))
            prod *= (1 - G_j)
        direction = mu3D[i] - cp
        c_i = build_color(sh[i], direction)
        C_values += c_i * G_i * prod
    return C_values

L1 = torch.tensor(0.0)
C = []
for k in tqdm(range(M)):
    C.append( C_(p_k[k]) )
    L1 += ( C[-1] - C_hat[k])**2
C = torch.stack(C)

L2 = dssim_loss( C , C_hat)

L = L2 #+L2
print("L=", L)

# Backward pass
L.backward()

# Gradients
print(mu3D.grad)
print(s.grad)
print(sh.grad)


  0%|          | 0/4 [00:00<?, ?it/s]

L= tensor(0.9929, grad_fn=<RsubBackward1>)
tensor([[ 0.0124,  0.0126, -0.0112],
        [ 0.0086,  0.0090, -0.0087],
        [ 0.0023,  0.0025, -0.0024]])
tensor([[-0.0077, -0.0085,  0.0000],
        [-0.0002, -0.0019,  0.0000],
        [ 0.0004, -0.0005,  0.0000]])
tensor([[-0.0068,  0.0119, -0.0119,  0.0119],
        [-0.0101,  0.0351, -0.0351,  0.0351],
        [-0.0042,  0.0218, -0.0218,  0.0218]])


#### Comparação:

In [93]:
dL_dmu = torch.tensor(dL_dmu, dtype=torch.float32).reshape(mu3D.grad.shape)
dL_ds = torch.tensor(dL_ds, dtype=torch.float32).reshape(s.grad.shape)
dL_dsh = torch.tensor(dL_dsh, dtype=torch.float32).reshape(sh.grad.shape)

# Gradients
print(mu3D.grad, "=\n", dL_dmu, "\n")
print(s.grad, "=\n", dL_ds, "\n" )
print(sh.grad, "=\n", dL_dsh, "\n" )

tensor([[ 0.0124,  0.0126, -0.0112],
        [ 0.0086,  0.0090, -0.0087],
        [ 0.0023,  0.0025, -0.0024]]) =
 tensor([[ 0.0124,  0.0126, -0.0112],
        [ 0.0086,  0.0090, -0.0087],
        [ 0.0023,  0.0025, -0.0024]]) 

tensor([[-0.0077, -0.0085,  0.0000],
        [-0.0002, -0.0019,  0.0000],
        [ 0.0004, -0.0005,  0.0000]]) =
 tensor([[-0.0077, -0.0085,  0.0000],
        [-0.0002, -0.0019,  0.0000],
        [ 0.0004, -0.0005,  0.0000]]) 

tensor([[-0.0068,  0.0119, -0.0119,  0.0119],
        [-0.0101,  0.0351, -0.0351,  0.0351],
        [-0.0042,  0.0218, -0.0218,  0.0218]]) =
 tensor([[-0.0068,  0.0119, -0.0119,  0.0119],
        [-0.0101,  0.0351, -0.0351,  0.0351],
        [-0.0042,  0.0218, -0.0218,  0.0218]]) 

